### 函数

#### import

In [1]:
import torch
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 使用第二块GPU（从0开始）
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import random
import torch.nn.functional as F
import torchvision
import numpy as np
from math import *
import matplotlib.pyplot as plt
from torch.autograd import Variable
from IPython import display
import torch.utils.data as Data
import torch.nn as nn
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import cm
from scipy.linalg import block_diag
import datetime
from torch.nn.utils import *
import cvxpy as cp
from Transformer_model import *
from RIS import *


import warnings
warnings.filterwarnings('ignore')

#### 参数

In [5]:
fc = 1.5e11; #150GHz  

# BW = 1e9;    # 1GHz

BW = 1e10;    # 10GHz  

Nc = 64;     # 64个子载波  


M = 2; #M×M子阵列行列数量
N = 8; #每个子阵N×N天线
c = 3e8;  
lambda_c = c/fc;  
D_sub = 100*lambda_c;  #子阵中心间隔 0.2m 
D_ant = lambda_c/2;    #天线间隔     2mm  
R = M*D_sub**2/lambda_c;#距离         40m   满足LOS-MIMO距离  

M_ant = M*M*N*N;

h1 = 10;  #BS与RIS高度
h2 = 1.5; #用户高度  此项作废，因为改为了每个散射体0-4米之间分布

L  = 8  #观测时隙数
Lp  = 1  #多径数
sigma = 1e5
B = 128

K = M*M




In [6]:
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
H_BR = Channel_BS_RIS(param_list) * 1e6

#### 定义csiNet

In [ ]:

def mish(x):
    x = x * (torch.tanh(F.softplus(x)))
    return x
class Mish(nn.Module):
    def __init__(self):
        super().__init__()
#         print("Mish activation loaded...")
    def forward(self,x):
        x = x * (torch.tanh(F.softplus(x)))
        return x
class RES_BLOCK(nn.Module): #输入信道输出 量化后的B比特反馈信息
    def __init__(self,channel_list):
        super(RES_BLOCK,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels = channel_list[0], #图片通道数
                out_channels = channel_list[1],#filter数量
                kernel_size = (5,5), #filter大小
                stride=1,  #扫描步进
                padding=(2,2), #周围围上2圈0 使得输出的宽和高和之前一样不变小
            ),
            nn.BatchNorm2d(channel_list[1], eps=1e-05, momentum=0.1, affine=True),
            Mish(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels = channel_list[1], #图片通道数
                out_channels = channel_list[2],#filter数量
                kernel_size = (5,5), #filter大小
                stride=1,  #扫描步进
                padding=(2,2), #周围围上2圈0 使得输出的宽和高和之前一样不变小
            ),
            nn.BatchNorm2d(channel_list[2], eps=1e-05, momentum=0.1, affine=True),
            Mish(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels = channel_list[2], #图片通道数
                out_channels = channel_list[0],#filter数量
                kernel_size = (5,5), #filter大小
                stride=1,  #扫描步进
                padding=(2,2), #周围围上2圈0 使得输出的宽和高和之前一样不变小
            ),
            nn.BatchNorm2d(channel_list[0], eps=1e-05, momentum=0.1, affine=True),
        )
        
        
    def forward(self,x_ini):
        
        
        x = self.conv1(x_ini)
        x = self.conv2(x)
        x = self.conv3(x)
        x = mish(x+x_ini)
        return x

def conv7x7(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=7, stride=stride,
                     padding=3, bias=True)

In [ ]:
#csiNet
class CSI_EN(nn.Module): #单层
    def __init__(self, param_list): #H [batch,G,K]为角度域信道, A [M,G]为观测矩阵 G为原始稀疏数据长度，T为迭代次数，K为子载波数也就是MMV的数量，M为观测维度
        super(CSI_EN,self).__init__()
        
#         param_list = [fc,B,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L]

        fc = param_list[0]
        BW  = param_list[1]
        Nc = param_list[2]
        M  = param_list[3]
        N  = param_list[4]
        D_sub = param_list[5]
        D_ant = param_list[6]
        R = param_list[7]
        M_ant = param_list[8]
        h1 = param_list[9]
        h2 = param_list[10]
        L =  param_list[11]
        sigma = param_list[12]
        B = param_list[13]
        

        


        #############
        self.B_quan = 2
        self.res1  = RES_BLOCK([2,16,32])
        self.res2  = RES_BLOCK([2,16,32])
        self.FC = nn.Linear(2*Nc*M_ant,B//self.B_quan)#全连接
        self.bn = nn.BatchNorm1d(B//self.B_quan)
        self.QL = QuantizationLayer(self.B_quan)
        
        
        
        
        
    def forward(self, param_list, H_BR, H_RU):
        #H_BR [ M_ant, M_ant]  H_RU [batch,Nc,M_ant]
        # param_list = [fc,B,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,SNR]
        fc = param_list[0]
        B  = param_list[1]
        Nc = param_list[2]
        M  = param_list[3]
        N  = param_list[4]
        D_sub = param_list[5]
        D_ant = param_list[6]
        R = param_list[7]
        M_ant = param_list[8]
        h1 = param_list[9]
        h2 = param_list[10]
        L =  param_list[11]
        sigma = param_list[12]
        B = param_list[13]
        
        H_real = torch.zeros(H_RU.shape[0],2,Nc,M_ant).cuda()#总观测【batch,Nc,L】
        H_real[:,0,:,:] = torch.real(H_RU)
        H_real[:,1,:,:] = torch.imag(H_RU)
        
        x = (self.res1(H_real))
        x = (self.res2(x))


        
        x = x.reshape(-1,2*Nc*M_ant)
        x = self.FC(x)
        x = self.bn(x)
        x = torch.sigmoid(x)
        x = self.QL(x)
        return x

class CSI_DE(nn.Module): #单层
    def __init__(self, param_list): #H [batch,G,K]为角度域信道, A [M,G]为观测矩阵 G为原始稀疏数据长度，T为迭代次数，K为子载波数也就是MMV的数量，M为观测维度
        super(CSI_DE,self).__init__()
        
#         param_list = [fc,B,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L]

        fc = param_list[0]
        BW  = param_list[1]
        Nc = param_list[2]
        M  = param_list[3]
        N  = param_list[4]
        D_sub = param_list[5]
        D_ant = param_list[6]
        R = param_list[7]
        M_ant = param_list[8]
        h1 = param_list[9]
        h2 = param_list[10]
        L =  param_list[11]
        sigma = param_list[12]
        B = param_list[13]
        
        self.B_quan = 2
        self.DQL = DequantizationLayer(self.B_quan)
        self.FC1 = nn.Linear(B//self.B_quan,2*Nc*M_ant)#全连接
        self.bn1 = nn.BatchNorm1d(2*Nc*M_ant)
        
        
        self.res1  = RES_BLOCK([2,16,32])
        self.res2  = RES_BLOCK([2,16,32])
        self.res3  = RES_BLOCK([2,16,32])
        self.res4  = RES_BLOCK([2,16,32])
        self.out_cov = conv7x7(2, 2)
        
        
        
        
    def forward(self, param_list,  x):
        #H_BR [ M_ant, M_ant]  H_RU [batch,Nc,M_ant]
        # param_list = [fc,B,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,SNR]
        fc = param_list[0]
        B  = param_list[1]
        Nc = param_list[2]
        M  = param_list[3]
        N  = param_list[4]
        D_sub = param_list[5]
        D_ant = param_list[6]
        R = param_list[7]
        M_ant = param_list[8]
        h1 = param_list[9]
        h2 = param_list[10]
        L =  param_list[11]
        sigma = param_list[12]
        B = param_list[13]
        
        x = self.DQL(x)-0.5
        x = self.FC1(x)
        x = self.bn1(x)
        x = mish(x)
        
        x = x.reshape(-1,2,Nc,M_ant)
        x = (self.res1(x))
        x = (self.res2(x))
        x = (self.res3(x))
        x = (self.res4(x))
        x = self.out_cov(x)
        
        H_RU = x[:,0,:,:] + 1j*x[:,1,:,:]
        return H_RU

class csiNet(nn.Module): #单层
    def __init__(self, param_list): #H [batch,G,K]为角度域信道, A [M,G]为观测矩阵 G为原始稀疏数据长度，T为迭代次数，K为子载波数也就是MMV的数量，M为观测维度
        super(csiNet,self).__init__()
        self.encoder = CSI_EN(param_list)
        self.decoder = CSI_DE(param_list)
        
        
    def forward(self, param_list,H_BR, H_RU1):
        
        out = self.encoder(param_list,H_BR,H_RU1)
        H_RU2 = self.decoder(param_list,out)
        #H_BR [ M_ant, M_ant]  H_RU [batch,Nc,M_ant]
        # param_list = [fc,B,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,SNR]

        return H_RU2

In [ ]:
def train_csiNet(param_list,num_batch, batch,STEPS,model_name_AE):
    
    fc = param_list[0]
#     B  = param_list1[1]
#     Nc = param_list1[2]
    M  = param_list[3]
    N  = param_list[4]
    D_sub = param_list[5]
    D_ant = param_list[6]
    R = param_list[7]
    M_ant = param_list[8]
    h1 = param_list[9]
    h2 = param_list[10]
    L =  param_list[11]
    sigma = param_list[12]
    B = param_list[13]
    Lp = param_list[15]

    net = csiNet(param_list).cuda()
        
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),lr=0.001)
    
    Ay = (M-1)*D_sub + (N-1)*(D_ant);#阵列长度
    Az = Ay;

    r_BS  = (torch.tensor([0,0,h1]) - torch.tensor([0,Ay/2,Az/2])).cuda()
    r_RIS = (torch.tensor([R,0,h1]) - torch.tensor([0,Ay/2,Az/2])).cuda()

    
    ##随机生成信道
    # H_BR = Channel_BS_RIS(param_list) * 1e6
    
    
    
    num_train = 0
    train_nmse = 0
    best_nmse = 1
    start = datetime.datetime.now()
    numStep = 1
    warmup_steps = 4000
    d_model = 256
    for step in range(STEPS):
        # optimizer.param_groups[0]['lr'] = (d_model**(-0.5))*min(numStep**(-0.5),numStep*warmup_steps**(-1.5))
        numStep = numStep+1
        net.train()
        optimizer.zero_grad()
        for num_b in range(num_batch):

            alpha = 1e5
            H_RU = Channel_RIS_UE(param_list,batch,alpha)


            H_RU0 = net(param_list, H_BR, H_RU)        
            loss = NMSE(H_RU0,H_RU)/num_batch
            loss.backward()
            train_nmse = train_nmse + loss

        optimizer.step()
        optimizer.zero_grad()
        num_train = num_train + 1
        
        if (step+1)%50==0:
            with torch.no_grad():
                net.eval()
                alpha = 1e5
                H_RU = Channel_RIS_UE(param_list,batch,alpha)
        
                H_RU0 = net(param_list, H_BR, H_RU)
        
                
                loss = NMSE(H_RU0,H_RU)
#                 print(H_BR)
#                 print(H_RU)

                
#                 print(R_UE0)
#                 print(R_UE)
#                 print(Theta_UE0)
#                 print(Theta_UE)
                
#                 r_UE0 = net(param_list, H_BR, H_RU)
#                 loss = loss_func(r_UE0, r_UE)
#             print(r_UE0)
#             print(r_UE)
            
            train_nmse = train_nmse/num_train
            time0 =  datetime.datetime.now()-start
            print('step:',step,'time',time0,'train NMSE %.5f' % train_nmse.cpu(),'test NMSE %.5f' % loss.cpu()) 
            if loss < best_nmse:
                best_nmse = loss
                torch.save(net, model_name_AE)
                print('Model saved!')
            num_train = 0
            train_nmse = 0
            start = datetime.datetime.now()

#### 压缩感知算法

In [7]:
def SOMP_GMMV(A,Y,K1,K2): #A为感知矩阵[M,L,N]，Y为观测信号矩阵[batch,M,L,1]，K1为用户数，K2为稀疏度,恢复稀疏信号X[batch,M,N,1]
    #Y为观测信号矩阵[batch,L,M]，K为稀疏度,恢复稀疏信号X[batch,N,M]
    batch = Y.shape[0]
    N = A.shape[2] #稀疏信号总长度
    M = Y.shape[1] #MMV数
    N_sub = N//K1  #子阵稀疏信号
    L = Y.shape[2] #压缩观测数
    R = Y #残差 [batch,M,L,1]
    ind_set = torch.floor(torch.zeros(batch,K2,1)).cuda().long()

    AA = A.repeat(batch, 1, 1 ,1) #取索引的时候gather需要输入每一个样本的 #[batch,M,L,N_sub]

    for a in range(K2):
        C = Hermitian(A) @ R #[batch,M,N,1]
        C = C.reshape(batch,M,N)
        C2 = Hermitian(C) @ C
        D = torch.abs(torch.diagonal(C2,dim1=-2, dim2=-1))
        ind = torch.max(D, 1)[1].reshape(-1,1,1,1)   #[batch,1,1,1]
        ind_set[:,a,:] = ind.reshape(-1,1)
        if a == 0:
            Supp = torch.gather(AA, 3, ind.repeat(1, M, L, 1))  #按照batch取索引,获得支撑集[batch,M,L,1],最终为[batch,M,L,K]
        else:
            Supp = torch.cat([Supp,torch.gather(AA, 3, ind.repeat(1, M, L, 1))],3)
        X_LS = torch.pinverse(Supp) @ Y #[batch,M,K,1]
        R = Y - Supp @ X_LS
    X_hat = torch.zeros(batch,M,N,1).cuda() + 0j
    for b in range(batch):  #最终的输出X_hat [batch, N, M]
        for k in range(K2):
            X_hat[b,:,ind_set[b,k,0],:] = X_LS[b,:,k,:]
    return X_hat

In [8]:
#考虑不同子载波不同观测
def SS_OMP_GMMV(A,Y,K1,K2): #A为感知矩阵[M,L,N]，Y为观测信号矩阵[batch,M,L,1]，K为稀疏度,恢复稀疏信号X[batch,M,N,1],SA为子阵数
    batch = Y.shape[0]
    N = A.shape[2] #稀疏信号总长度
    M = Y.shape[1] #MMV数
    N_sub = N//K1  #子阵稀疏信号
    L = Y.shape[2] #压缩观测数
    R = Y #残差 [batch,M,L,1]
    ind_set = torch.floor(torch.zeros(batch,K2,1)).cuda().long()
    for b in range(K2//K1):
        for a in range(K1):
            AA_k = A[:,:,(a*N_sub):((a+1)*N_sub)] #[M,L,N_sub]
            C = Hermitian(AA_k) @ R #[batch,M,N_sub,1]
            C = C.reshape(batch,M,N_sub)
            if a == 0:
                C2 = Hermitian(C) @ C
            else:
                C2 = C2 + Hermitian(C) @ C

        
        D = torch.abs(torch.diagonal(C2,dim1=-2, dim2=-1))
        for a in range(K):
            AA_k = A[:,:,(a*N_sub):((a+1)*N_sub)] #[M,L,N_sub]
            ind = torch.max(D, 1)[1].reshape(-1,1,1,1)   #[batch,1,1,1]
            ind_set[:,b*K1+a,:] = ind.reshape(-1,1) + a*N_sub
            AA_k = AA_k.repeat(batch, 1, 1 ,1) #取索引的时候gather需要输入每一个样本的 #[batch,M,L,N_sub]
            if b*K1+a == 0:
                Supp = torch.gather(AA_k, 3, ind.repeat(1, M, L, 1))  #按照batch取索引,获得支撑集[batch,M,L,1],最终为[batch,M,L,K]
            else:
                Supp = torch.cat([Supp,torch.gather(AA_k, 3, ind.repeat(1, M, L, 1)) ],3) #[batch,M,L,K]
        X_LS = torch.pinverse(Supp) @ Y #[batch,M,K,1]
        R = Y - Supp @ X_LS

    X_hat = torch.zeros(batch,M,N,1).cuda() + 0j
    for b in range(batch):  #最终的输出X_hat [batch, M, N, 1]
        for k in range(K):
            X_hat[b,:,ind_set[b,k,0],:] = X_LS[b,:,k,:]
    return X_hat

In [9]:
def sigmoid(x):
    return torch.sigmoid(torch.from_numpy(x)).numpy()

# MMV-AMP algorithm Channel Estimation
def AMP_GMMV_Meng(y, H, IteN):

# y [N_RAs,N_dim] 观测值
# H [N_dim,N_RAs,N_UEs] 观测矩阵

    N_RAs = H.shape[1]   #观测维度
    N_UEs = H.shape[2]   #稀疏维度
    N_dim = y.shape[1]   #MMV维度

    y = y.astype(np.complex128)
    H = H.astype(np.complex128)
 
    import scipy.stats as st
    import math
    lam = N_RAs/N_UEs
    c = np.arange(0.01, 10, 10/1024)
    rho = (1 - 2 * N_UEs * ((1 + c**2) * st.norm.cdf(-c)-c * st.norm.pdf(c))/N_RAs)/(1 + c**2 - 2 * ((1 + c**2) * st.norm.cdf(-c) - c * st.norm.pdf(c)))
    alpha = lam * np.amax(rho) * np.ones((N_UEs, N_dim)) ##[N_UEs, N_dim]

    x_hat = np.zeros((N_UEs, N_dim), dtype='complex128')  ##[N_UEs, N_dim]
    var_hat = np.ones((N_UEs, N_dim), dtype='float64')   ##[N_UEs, N_dim]

    V = np.ones((N_RAs, N_dim))  ##[N_RAs, N_dim]
    Z = y + 0 ##[N_RAs, N_dim]
    SNR0 = 100
    sigma2 = np.linalg.norm(y)**2/((SNR0 + 1) * N_RAs * N_dim)  ##1
    hvar = (np.linalg.norm(y)**2 - N_RAs * sigma2)/(N_dim * lam * np.amax(rho) * ((np.linalg.norm(H)**2)/N_dim) )  ##1
    hmean = 0  ##1

    pai = np.zeros((N_UEs, N_dim), dtype='float64')   ##

    t = 1
    tol = 1e-5
    damp = 0.3

    while t < IteN:
        x_hat_new = np.zeros((N_UEs, N_dim), dtype='complex128')  ##[N_UEs, N_dim]
        x_hat_pre = np.zeros((N_UEs, N_dim), dtype='complex128')  ##[N_UEs, N_dim]
        Vi_tol = np.zeros((N_UEs, N_dim), dtype='float64')  ##[N_UEs, N_dim]
        B_tol = np.zeros((N_UEs, N_dim), dtype='complex128')  ##[N_UEs, N_dim]
        for a in range(N_dim):
            x_hat_pre[:,a:a+1] = x_hat[:,a:a+1]
            V_new = np.dot(np.abs(H[a])*np.abs(H[a]), var_hat[:,a:a+1])
            Z_new = np.dot(H[a], x_hat[:,a:a+1]) - ((y[:,a:a+1] - Z[:,a:a+1]) / (sigma2 + V[:,a:a+1])) * V_new   # + 1e-8

            Z_new = damp * Z[:,a:a+1] + (1 - damp) * Z_new
            V_new = damp * V[:,a:a+1] + (1 - damp) * V_new

            var1 = np.dot((np.abs(H[a])*np.abs(H[a])).T, 1/(sigma2 + V_new))
            var2 = np.dot(H[a].conj().T, (y[:,a:a+1] - Z_new)/(sigma2 + V_new))

            Ri = var2/(var1) + x_hat[:,a:a+1]
            Vi = 1/(var1)
            Vi_tol[:,a:a+1] = Vi

            # A
            A = (hvar * Vi)/(Vi + hvar)
            # B
            B = (hvar * Ri + Vi * hmean)/(Vi + hvar)
            B_tol[:,a:a+1] = B
            # lll
            lll = np.log(Vi/(Vi + hvar))/2 + np.abs(Ri)**2/2/Vi - np.abs(Ri - hmean)**2/2/(Vi + hvar)
            # pai
            # pai[:,a:a+1] = alpha[:,a:a+1]/(alpha[:,a:a+1] + (1 - alpha[:,a:a+1]) * np.exp(-lll))
            pai[:,a:a+1] = sigmoid(lll-np.log((1-alpha[:,a:a+1])/alpha[:,a:a+1]))

            x_hat_new[:,a:a+1] = pai[:,a:a+1] * B

            var_hat_new = pai[:,a:a+1]*(np.abs(B)**2 + Vi) - np.abs(x_hat_new[:,a:a+1])**2

            x_hat[:,a:a+1] = x_hat_new[:,a:a+1]
            var_hat[:,a:a+1] = var_hat_new
            
            V[:,a:a+1] = V_new
            # Z[:,a:a+1] = Z_new + 0
            # Z = 0
            Z[:,a:a+1] = Z_new.copy() + 0

        # 噪声方差 自学习
        sigma2 = ((np.abs(y-Z)**2)/(np.abs(1+V/sigma2)**2)+sigma2*V/(V+sigma2)).mean()
        # 先验均值 自学习
        hmean = (np.sum(pai * B_tol, axis=0)/np.sum(pai, axis=0)).mean()
        # 先验方差 自学习
        hvar = (np.sum(pai * (np.abs(hmean - B_tol)**2 + Vi_tol), axis=0)/np.sum(pai, axis=0)).mean()
        # 活跃因子 EM学习
        pai[pai < 1e-5] = 1e-5
        pai[pai > 1 - 1e-5] = 1 - 1e-5
        alpha_new00 = (np.sum(pai, axis=1)/N_dim).reshape(-1, 1)

        ###取平均

        alpha_new = np.tile(alpha_new00, (1, N_dim))
        alpha = alpha_new

        NMSE_iter = np.sum(np.abs(x_hat_pre - x_hat_new)**2)/np.sum(np.abs(x_hat_new)**2)
        if NMSE_iter < tol:
            break
        xx = x_hat.transpose(1,0).reshape(N_dim,N_UEs,1)
        y_hat = H @ xx #[N_dim,N_RAs,1]
        y_hat = y_hat.transpose(1,0,2).reshape(N_RAs,N_dim)
        NMSE_iter = np.sum(np.abs(y_hat - y)**2)/np.sum(np.abs(y)**2)
        if NMSE_iter > 10:
            x_hat = x_hat - x_hat
            break


        t = t + 1
    return x_hat, var_hat, alpha, t
# MMV-AMP algorithm Channel Estimation
def BAMP_GMMV_Meng(y, H, IteN):

# y [N_RAs,N_dim] 观测值
# H [N_dim,N_RAs,N_UEs] 观测矩阵

    N_RAs = H.shape[1]   #观测维度
    N_UEs = H.shape[2]   #稀疏维度
    N_dim = y.shape[1]   #MMV维度

    y = y.astype(np.complex128)
    H = H.astype(np.complex128)
 
    import scipy.stats as st
    import math
    lam = N_RAs/N_UEs
    c = np.arange(0.01, 10, 10/1024)
    rho = (1 - 2 * N_UEs * ((1 + c**2) * st.norm.cdf(-c)-c * st.norm.pdf(c))/N_RAs)/(1 + c**2 - 2 * ((1 + c**2) * st.norm.cdf(-c) - c * st.norm.pdf(c)))
    alpha = lam * np.amax(rho) * np.ones((N_UEs, N_dim)) ##[N_UEs, N_dim]

    x_hat = np.zeros((N_UEs, N_dim), dtype='complex128')  ##[N_UEs, N_dim]
    var_hat = np.ones((N_UEs, N_dim), dtype='float64')   ##[N_UEs, N_dim]

    V = np.ones((N_RAs, N_dim))  ##[N_RAs, N_dim]
    Z = y + 0 ##[N_RAs, N_dim]
    SNR0 = 100
    sigma2 = np.linalg.norm(y)**2/((SNR0 + 1) * N_RAs * N_dim)  ##1
    hvar = (np.linalg.norm(y)**2 - N_RAs * sigma2)/(N_dim * lam * np.amax(rho) * ((np.linalg.norm(H)**2)/N_dim) )  ##1
    hmean = 0  ##1

    pai = np.zeros((N_UEs, N_dim), dtype='float64')   ##

    t = 1
    tol = 1e-5
    damp = 0.3

    while t < IteN:
        x_hat_new = np.zeros((N_UEs, N_dim), dtype='complex128')  ##[N_UEs, N_dim]
        x_hat_pre = np.zeros((N_UEs, N_dim), dtype='complex128')  ##[N_UEs, N_dim]
        Vi_tol = np.zeros((N_UEs, N_dim), dtype='float64')  ##[N_UEs, N_dim]
        B_tol = np.zeros((N_UEs, N_dim), dtype='complex128')  ##[N_UEs, N_dim]
        for a in range(N_dim):
            x_hat_pre[:,a:a+1] = x_hat[:,a:a+1]
            V_new = np.dot(np.abs(H[a])*np.abs(H[a]), var_hat[:,a:a+1])
            Z_new = np.dot(H[a], x_hat[:,a:a+1]) - ((y[:,a:a+1] - Z[:,a:a+1]) / (sigma2 + V[:,a:a+1])) * V_new   # + 1e-8

            Z_new = damp * Z[:,a:a+1] + (1 - damp) * Z_new
            V_new = damp * V[:,a:a+1] + (1 - damp) * V_new

            var1 = np.dot((np.abs(H[a])*np.abs(H[a])).T, 1/(sigma2 + V_new))
            var2 = np.dot(H[a].conj().T, (y[:,a:a+1] - Z_new)/(sigma2 + V_new))

            Ri = var2/(var1) + x_hat[:,a:a+1]
            Vi = 1/(var1)
            Vi_tol[:,a:a+1] = Vi

            # A
            A = (hvar * Vi)/(Vi + hvar)
            # B
            B = (hvar * Ri + Vi * hmean)/(Vi + hvar)
            B_tol[:,a:a+1] = B
            # lll
            lll = np.log(Vi/(Vi + hvar))/2 + np.abs(Ri)**2/2/Vi - np.abs(Ri - hmean)**2/2/(Vi + hvar)
            # pai
            # pai[:,a:a+1] = alpha[:,a:a+1]/(alpha[:,a:a+1] + (1 - alpha[:,a:a+1]) * np.exp(-lll))
            pai[:,a:a+1] = sigmoid(lll-np.log((1-alpha[:,a:a+1])/alpha[:,a:a+1]))

            x_hat_new[:,a:a+1] = pai[:,a:a+1] * B

            var_hat_new = pai[:,a:a+1]*(np.abs(B)**2 + Vi) - np.abs(x_hat_new[:,a:a+1])**2

            x_hat[:,a:a+1] = x_hat_new[:,a:a+1]
            var_hat[:,a:a+1] = var_hat_new
            
            V[:,a:a+1] = V_new
            # Z[:,a:a+1] = Z_new + 0
            # Z = 0
            Z[:,a:a+1] = Z_new.copy() + 0

        # 噪声方差 自学习
        sigma2 = ((np.abs(y-Z)**2)/(np.abs(1+V/sigma2)**2)+sigma2*V/(V+sigma2)).mean()
        # 先验均值 自学习
        hmean = (np.sum(pai * B_tol, axis=0)/np.sum(pai, axis=0)).mean()
        # 先验方差 自学习
        hvar = (np.sum(pai * (np.abs(hmean - B_tol)**2 + Vi_tol), axis=0)/np.sum(pai, axis=0)).mean()
        # 活跃因子 EM学习
        pai[pai < 1e-5] = 1e-5
        pai[pai > 1 - 1e-5] = 1 - 1e-5
        alpha_new00 = (np.sum(pai, axis=1)/N_dim).reshape(-1, 1)

        ###取平均

        alpha_new = np.tile(alpha_new00, (1, N_dim))
        alpha = alpha_new

        alpha = alpha.reshape(K, N_UEs//K, N_dim)
        alpha = alpha.mean(axis=0)
        alpha = np.tile(alpha, (K, 1))



        NMSE_iter = np.sum(np.abs(x_hat_pre - x_hat_new)**2)/np.sum(np.abs(x_hat_new)**2)
        if NMSE_iter < tol:
            break
        xx = x_hat.transpose(1,0).reshape(N_dim,N_UEs,1)
        y_hat = H @ xx #[N_dim,N_RAs,1]
        y_hat = y_hat.transpose(1,0,2).reshape(N_RAs,N_dim)
        NMSE_iter = np.sum(np.abs(y_hat - y)**2)/np.sum(np.abs(y)**2)
        if NMSE_iter > 10:
            x_hat = x_hat - x_hat
            break


        t = t + 1
    return x_hat, var_hat, alpha, t

# MMV-AMP algorithm Channel Estimation
def AMP_GMMV(H, y, IteN):#A为感知矩阵[M,L,N]，Y为观测信号矩阵[batch,M,L,1]，K为稀疏度,恢复稀疏信号X[batch,M,N,1],SA为子阵数
    L = H.shape[1]   #观测维度
    N = H.shape[2]   #稀疏维度
    M = y.shape[1]   #MMV维度
    batch = y.shape[0]
    X_hat = torch.zeros(batch,N,M).cuda() + 0j#[batch,N,M]
    for i in range(batch):
        yy = y[i].reshape(M,L)
        yy = yy.permute(1,0) #[L,M]
        a,b,c,d = AMP_GMMV_Meng(yy.cpu().numpy(),H.cpu().numpy(),20) # Y为观测信号矩阵[batch,L,M]
        # a,b,c,d = AMP_MMV_Meng(yy.cpu().numpy(),H[0].cpu().numpy(),50) # Y为观测信号矩阵[batch,L,M]
        X_hat[i] = torch.from_numpy(a).cuda()
    X_hat = X_hat.permute(0,2,1)
    X_hat = X_hat.reshape(batch,M,N,1)
    return X_hat
def BAMP_GMMV(H, y, IteN):#A为感知矩阵[M,L,N]，Y为观测信号矩阵[batch,M,L,1]，K为稀疏度,恢复稀疏信号X[batch,M,N,1],SA为子阵数
    L = H.shape[1]   #观测维度
    N = H.shape[2]   #稀疏维度
    M = y.shape[1]   #MMV维度
    batch = y.shape[0]
    X_hat = torch.zeros(batch,N,M).cuda() + 0j#[batch,N,M]
    for i in range(batch):
        yy = y[i].reshape(M,L)
        yy = yy.permute(1,0) #[L,M]
        a,b,c,d = BAMP_GMMV_Meng(yy.cpu().numpy(),H.cpu().numpy(),20) # Y为观测信号矩阵[batch,L,M]
        # a,b,c,d = AMP_MMV_Meng(yy.cpu().numpy(),H[0].cpu().numpy(),50) # Y为观测信号矩阵[batch,L,M]
        X_hat[i] = torch.from_numpy(a).cuda()
    X_hat = X_hat.permute(0,2,1)
    X_hat = X_hat.reshape(batch,M,N,1)
    return X_hat

In [10]:
def CE_SOMP_GMMV(param_list,batch,pho):
    #Phi 为观测矩阵 Theta为感知矩阵

    fc = param_list[0]
    BW  = param_list[1]
    Nc = param_list[2]
    M  = param_list[3]
    N  = param_list[4]
    D_sub = param_list[5]
    D_ant = param_list[6]
    R = param_list[7]
    M_ant = param_list[8]
    h1 = param_list[9]
    h2 = param_list[10]
    L =  param_list[11]
    sigma = param_list[12]
    B = param_list[13]
    K = param_list[14]

    N_2 = pho*N 

    alpha = 1e5
    # H_RU = H_RU.reshape(batch,Nc,M_ant,1)
    Step = 1

    K1 = K
    K2 = K


    NMSE = 0

    for step in range(Step):
        A = torch.zeros(N*N,N_2*N_2).cuda() + 0j
        A_t = torch.zeros(M_ant,N_2*N_2*M*M).cuda() + 0j
        for i in range(N_2):
            for j in range(N_2):
                a_h = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*i*2*pi
                a_v = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*j*2*pi
                a = kron_add(a_h,a_v).reshape(N*N)
                A[:,i*N_2+j] = torch.exp(1j*a)
        for i in range(M*M):
            A_t[i*N*N:(i+1)*N*N,i*N_2*N_2:(i+1)*N_2*N_2] = A
        H_RU = Channel_RIS_UE(param_list,batch,alpha)
        S_digital = torch.randn(L,Nc,4,1).cuda()+1j*torch.randn(L,Nc,4,1).cuda() #L个时隙BS导频,这里假设频率选择
        S_analog = torch.rand(L,1,M_ant,4).cuda() * 2*pi
        Phase = torch.rand(L,M_ant).cuda() * 2*pi   #L个时隙RIS相位

        Y = torch.zeros(H_RU.shape[0],Nc,L).cuda() + 0j#总观测【batch,Nc,L】

        Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j

        for l in range(L):
            s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(Nc,M_ant,1)  #导频s [Nc,M_ant]    
            
            s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
            s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(Nc))).reshape(1,1,1)
        #             print(s)
            n = (torch.randn(H_RU.shape[0],Nc) + 1j*torch.randn(H_RU.shape[0],Nc)).cuda()*sqrt(sigma/2)
            
        #             print(n)
            
            Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
            a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
            Phi[:,l,:] = a[:,0,:]
        #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
            y = (a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc) + n #[batch, Nc]
            Y[:,:,l] = y

        Theta = Phi @ A_t

        H_t_hat = SOMP_GMMV(Theta,Y.reshape(batch,Nc,L,1),K1,K2)
        H_hat = A_t @ H_t_hat
        NMSE = NMSE + ((torch.mean(torch.abs((H_RU-H_hat.reshape(batch,Nc,M_ant))*(H_RU-H_hat.reshape(batch,Nc,M_ant)))))/(torch.mean(torch.abs(H_RU*H_RU))))
    NMSE = NMSE/Step
    return H_RU,H_hat
    # print(NMSE)

def CE_SOMP_MMV(param_list,batch,pho):
    #Phi 为观测矩阵 Theta为感知矩阵
    fc = param_list[0]
    BW  = param_list[1]
    Nc = param_list[2]
    M  = param_list[3]
    N  = param_list[4]
    D_sub = param_list[5]
    D_ant = param_list[6]
    R = param_list[7]
    M_ant = param_list[8]
    h1 = param_list[9]
    h2 = param_list[10]
    L =  param_list[11]
    sigma = param_list[12]
    B = param_list[13]
    K = param_list[14]

    N_2 = pho*N 

    alpha = 1e5/sqrt(Lp)
    # H_RU = H_RU.reshape(batch,Nc,M_ant,1)
    Step = 1

    K1 = K
    K2 = K
    NMSE = 0

    for step in range(Step):
        A = torch.zeros(N*N,N_2*N_2).cuda() + 0j
        A_t = torch.zeros(M_ant,N_2*N_2*M*M).cuda() + 0j
        for i in range(N_2):
            for j in range(N_2):
                a_h = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*i*2*pi
                a_v = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*j*2*pi
                a = kron_add(a_h,a_v).reshape(N*N)
                A[:,i*N_2+j] = torch.exp(1j*a)
        for i in range(M*M):
            A_t[i*N*N:(i+1)*N*N,i*N_2*N_2:(i+1)*N_2*N_2] = A
        H_RU = Channel_RIS_UE(param_list,batch,alpha)
        S_digital = torch.randn(L,1,4,1).cuda()+1j*torch.randn(L,1,4,1).cuda() #L个时隙BS导频,这里假设频率选择
        S_analog = torch.rand(L,1,M_ant,4).cuda() * 2*pi
        Phase = torch.rand(L,M_ant).cuda() * 2*pi   #L个时隙RIS相位

        Y = torch.zeros(H_RU.shape[0],Nc,L).cuda() + 0j#总观测【batch,Nc,L】

        Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j

        for l in range(L):
            s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(1,M_ant,1)  #导频s [1,M_ant]    
            
            s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
            s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(1))).reshape(1,1,1)
        #             print(s)
            n = (torch.randn(H_RU.shape[0],Nc) + 1j*torch.randn(H_RU.shape[0],Nc)).cuda()*sqrt(sigma/2)
            
        #             print(n)
            
            Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
            a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
            Phi[:,l,:] = a[:,0,:]
        #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
            y = (a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc) + n #[batch, Nc]
            Y[:,:,l] = y

        Theta = Phi @ A_t

        H_t_hat = SOMP_GMMV(Theta,Y.reshape(batch,Nc,L,1),K1,K2)
        H_hat = A_t @ H_t_hat
        NMSE = NMSE + ((torch.mean(torch.abs((H_RU-H_hat.reshape(batch,Nc,M_ant))*(H_RU-H_hat.reshape(batch,Nc,M_ant)))))/(torch.mean(torch.abs(H_RU*H_RU))))
    NMSE = NMSE/Step
    return H_RU,H_hat
    # print(NMSE)

def CE_BSOMP_GMMV(param_list,batch,pho): #K1用户数 K2稀疏度
#Phi 为观测矩阵 Theta为感知矩阵
    fc = param_list[0]
    BW  = param_list[1]
    Nc = param_list[2]
    M  = param_list[3]
    N  = param_list[4]
    D_sub = param_list[5]
    D_ant = param_list[6]
    R = param_list[7]
    M_ant = param_list[8]
    h1 = param_list[9]
    h2 = param_list[10]
    L =  param_list[11]
    sigma = param_list[12]
    B = param_list[13]
    K = param_list[14]
    K1 = K
    K2 = K
    N_2 = pho*N 

    alpha = 1e5/sqrt(Lp)
    # H_RU = H_RU.reshape(batch,Nc,M_ant,1)
    Step = 1


    NMSE = 0

    for step in range(Step):
        A = torch.zeros(N*N,N_2*N_2).cuda() + 0j
        A_t = torch.zeros(M_ant,N_2*N_2*M*M).cuda() + 0j
        for i in range(N_2):
            for j in range(N_2):
                a_h = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*i*2*pi
                a_v = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*j*2*pi
                a = kron_add(a_h,a_v).reshape(N*N)
                A[:,i*N_2+j] = torch.exp(1j*a)
        for i in range(M*M):
            A_t[i*N*N:(i+1)*N*N,i*N_2*N_2:(i+1)*N_2*N_2] = A
        H_RU = Channel_RIS_UE(param_list,batch,alpha)

        S_digital = torch.randn(L,Nc,4,1).cuda()+1j*torch.randn(L,Nc,4,1).cuda() #L个时隙BS导频,这里假设频率选择
        S_analog = torch.rand(L,1,M_ant,4).cuda() * 2*pi
        Phase = torch.rand(L,M_ant).cuda() * 2*pi   #L个时隙RIS相位

        Y = torch.zeros(H_RU.shape[0],Nc,L).cuda() + 0j#总观测【batch,Nc,L】

        Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j

        for l in range(L):
            s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(Nc,M_ant,1)  #导频s [Nc,M_ant]    
            
            s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
            s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(Nc))).reshape(1,1,1)
        #             print(s)
            n = (torch.randn(H_RU.shape[0],Nc) + 1j*torch.randn(H_RU.shape[0],Nc)).cuda()*sqrt(sigma/2)
            
        #             print(n)
            
            Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
            a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
            Phi[:,l,:] = a[:,0,:]
        #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
            y = (a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc) + n #[batch, Nc]
            Y[:,:,l] = y

        Theta = Phi @ A_t

        H_t_hat = SS_OMP_GMMV(Theta,Y.reshape(batch,Nc,L,1),K1,K2)
        H_hat = A_t @ H_t_hat
        NMSE = NMSE + ((torch.mean(torch.abs((H_RU-H_hat.reshape(batch,Nc,M_ant))*(H_RU-H_hat.reshape(batch,Nc,M_ant)))))/(torch.mean(torch.abs(H_RU*H_RU))))
    NMSE = NMSE/Step
    return H_RU,H_hat
    # print(NMSE)


def CE_BSOMP_MMV(param_list,batch,pho): #K1用户数 K2稀疏度
#Phi 为观测矩阵 Theta为感知矩阵
    fc = param_list[0]
    BW  = param_list[1]
    Nc = param_list[2]
    M  = param_list[3]
    N  = param_list[4]
    D_sub = param_list[5]
    D_ant = param_list[6]
    R = param_list[7]
    M_ant = param_list[8]
    h1 = param_list[9]
    h2 = param_list[10]
    L =  param_list[11]
    sigma = param_list[12]
    B = param_list[13]
    K = param_list[14]
    K1 = K
    K2 = K
    N_2 = pho*N 

    alpha = 1e5/sqrt(Lp)
    # H_RU = H_RU.reshape(batch,Nc,M_ant,1)
    Step = 1


    NMSE = 0

    for step in range(Step):
        A = torch.zeros(N*N,N_2*N_2).cuda() + 0j
        A_t = torch.zeros(M_ant,N_2*N_2*M*M).cuda() + 0j
        for i in range(N_2):
            for j in range(N_2):
                a_h = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*i*2*pi
                a_v = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*j*2*pi
                a = kron_add(a_h,a_v).reshape(N*N)
                A[:,i*N_2+j] = torch.exp(1j*a)
        for i in range(M*M):
            A_t[i*N*N:(i+1)*N*N,i*N_2*N_2:(i+1)*N_2*N_2] = A
        H_RU = Channel_RIS_UE(param_list,batch,alpha)

        S_digital = torch.randn(L,1,4,1).cuda()+1j*torch.randn(L,1,4,1).cuda() #L个时隙BS导频,这里假设频率选择
        S_analog = torch.rand(L,1,M_ant,4).cuda() * 2*pi
        Phase = torch.rand(L,M_ant).cuda() * 2*pi   #L个时隙RIS相位

        Y = torch.zeros(H_RU.shape[0],Nc,L).cuda() + 0j#总观测【batch,Nc,L】

        Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j

        for l in range(L):
            s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(1,M_ant,1)  #导频s [Nc,M_ant]    
            
            s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
            s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(1))).reshape(1,1,1)
        #             print(s)
            n = (torch.randn(H_RU.shape[0],Nc) + 1j*torch.randn(H_RU.shape[0],Nc)).cuda()*sqrt(sigma/2)
            
        #             print(n)
            
            Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
            a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
            Phi[:,l,:] = a[:,0,:]
        #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
            y = (a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc) + n #[batch, Nc]
            Y[:,:,l] = y

        Theta = Phi @ A_t

        H_t_hat = SS_OMP_GMMV(Theta,Y.reshape(batch,Nc,L,1),K1,K2)
        H_hat = A_t @ H_t_hat
        NMSE = NMSE + ((torch.mean(torch.abs((H_RU-H_hat.reshape(batch,Nc,M_ant))*(H_RU-H_hat.reshape(batch,Nc,M_ant)))))/(torch.mean(torch.abs(H_RU*H_RU))))
    NMSE = NMSE/Step
    return H_RU,H_hat
    # print(NMSE)
def CE_AMP_MMV(param_list,batch,pho):
#Phi 为观测矩阵 Theta为感知矩阵
    fc = param_list[0]
    BW  = param_list[1]
    Nc = param_list[2]
    M  = param_list[3]
    N  = param_list[4]
    D_sub = param_list[5]
    D_ant = param_list[6]
    R = param_list[7]
    M_ant = param_list[8]
    h1 = param_list[9]
    h2 = param_list[10]
    L =  param_list[11]
    sigma = param_list[12]
    B = param_list[13]
    K = param_list[14]
    K1 = K
    K2 = K
    N_2 = pho*N 

    alpha = 1e5/sqrt(Lp)
    # H_RU = H_RU.reshape(batch,Nc,M_ant,1)
    Step = 1


    NMSE = 0

    for step in range(Step):
        A = torch.zeros(N*N,N_2*N_2).cuda() + 0j
        A_t = torch.zeros(M_ant,N_2*N_2*M*M).cuda() + 0j
        for i in range(N_2):
            for j in range(N_2):
                a_h = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*i*2*pi
                a_v = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*j*2*pi
                a = kron_add(a_h,a_v).reshape(N*N)
                A[:,i*N_2+j] = torch.exp(1j*a)
        for i in range(M*M):
            A_t[i*N*N:(i+1)*N*N,i*N_2*N_2:(i+1)*N_2*N_2] = A
        H_RU = Channel_RIS_UE(param_list,batch,alpha)

        S_digital = torch.randn(L,1,4,1).cuda()+1j*torch.randn(L,1,4,1).cuda() #L个时隙BS导频,这里假设频率选择
        S_analog = torch.rand(L,1,M_ant,4).cuda() * 2*pi
        Phase = torch.rand(L,M_ant).cuda() * 2*pi   #L个时隙RIS相位

        Y = torch.zeros(H_RU.shape[0],Nc,L).cuda() + 0j#总观测【batch,Nc,L】

        Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j

        for l in range(L):
            s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(1,M_ant,1)  #导频s [Nc,M_ant]    
            
            s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
            s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(1))).reshape(1,1,1)
        #             print(s)
            n = (torch.randn(H_RU.shape[0],Nc) + 1j*torch.randn(H_RU.shape[0],Nc)).cuda()*sqrt(sigma/2)
            
        #             print(n)
            
            Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
            a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
            Phi[:,l,:] = a[:,0,:]
        #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
            y = (a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc) + n #[batch, Nc]
            Y[:,:,l] = y

        Theta = Phi @ A_t

        H_t_hat = AMP_GMMV(Theta,Y.reshape(batch,Nc,L,1),K)
        H_hat = A_t @ H_t_hat
        NMSE = NMSE + ((torch.mean(torch.abs((H_RU-H_hat.reshape(batch,Nc,M_ant))*(H_RU-H_hat.reshape(batch,Nc,M_ant)))))/(torch.mean(torch.abs(H_RU*H_RU))))
    NMSE = NMSE/Step
    return H_RU,H_hat
    # print(NMSE)
def CE_BAMP_MMV(param_list,batch,pho):
#Phi 为观测矩阵 Theta为感知矩阵
    fc = param_list[0]
    BW  = param_list[1]
    Nc = param_list[2]
    M  = param_list[3]
    N  = param_list[4]
    D_sub = param_list[5]
    D_ant = param_list[6]
    R = param_list[7]
    M_ant = param_list[8]
    h1 = param_list[9]
    h2 = param_list[10]
    L =  param_list[11]
    sigma = param_list[12]
    B = param_list[13]
    K = param_list[14]
    K1 = K
    K2 = K
    N_2 = pho*N 

    alpha = 1e5/sqrt(Lp)
    # H_RU = H_RU.reshape(batch,Nc,M_ant,1)
    Step = 1


    NMSE = 0

    for step in range(Step):
        A = torch.zeros(N*N,N_2*N_2).cuda() + 0j
        A_t = torch.zeros(M_ant,N_2*N_2*M*M).cuda() + 0j
        for i in range(N_2):
            for j in range(N_2):
                a_h = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*i*2*pi
                a_v = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*j*2*pi
                a = kron_add(a_h,a_v).reshape(N*N)
                A[:,i*N_2+j] = torch.exp(1j*a)
        for i in range(M*M):
            A_t[i*N*N:(i+1)*N*N,i*N_2*N_2:(i+1)*N_2*N_2] = A
        H_RU = Channel_RIS_UE(param_list,batch,alpha)

        S_digital = torch.randn(L,1,4,1).cuda()+1j*torch.randn(L,1,4,1).cuda() #L个时隙BS导频,这里假设频率选择
        S_analog = torch.rand(L,1,M_ant,4).cuda() * 2*pi
        Phase = torch.rand(L,M_ant).cuda() * 2*pi   #L个时隙RIS相位

        Y = torch.zeros(H_RU.shape[0],Nc,L).cuda() + 0j#总观测【batch,Nc,L】

        Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j

        for l in range(L):
            s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(1,M_ant,1)  #导频s [Nc,M_ant]    
            
            s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
            s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(1))).reshape(1,1,1)
        #             print(s)
            n = (torch.randn(H_RU.shape[0],Nc) + 1j*torch.randn(H_RU.shape[0],Nc)).cuda()*sqrt(sigma/2)
            
        #             print(n)
            
            Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
            a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
            Phi[:,l,:] = a[:,0,:]
        #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
            y = (a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc) + n #[batch, Nc]
            Y[:,:,l] = y

        Theta = Phi @ A_t

        H_t_hat = BAMP_GMMV(Theta,Y.reshape(batch,Nc,L,1),K)
        H_hat = A_t @ H_t_hat
        NMSE = NMSE + ((torch.mean(torch.abs((H_RU-H_hat.reshape(batch,Nc,M_ant))*(H_RU-H_hat.reshape(batch,Nc,M_ant)))))/(torch.mean(torch.abs(H_RU*H_RU))))
    NMSE = NMSE/Step
    return H_RU,H_hat
    # print(NMSE)

def CE_BAMP_GMMV(param_list,batch,pho):
#Phi 为观测矩阵 Theta为感知矩阵

    fc = param_list[0]
    BW  = param_list[1]
    Nc = param_list[2]
    M  = param_list[3]
    N  = param_list[4]
    D_sub = param_list[5]
    D_ant = param_list[6]
    R = param_list[7]
    M_ant = param_list[8]
    h1 = param_list[9]
    h2 = param_list[10]
    L =  param_list[11]
    sigma = param_list[12]
    B = param_list[13]
    K = param_list[14]
    K1 = K
    K2 = K
    N_2 = pho*N 

    alpha = 1e5/sqrt(Lp)
    # H_RU = H_RU.reshape(batch,Nc,M_ant,1)
    Step = 1


    NMSE = 0

    for step in range(Step):
        A = torch.zeros(N*N,N_2*N_2).cuda() + 0j
        A_t = torch.zeros(M_ant,N_2*N_2*M*M).cuda() + 0j
        for i in range(N_2):
            for j in range(N_2):
                a_h = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*i*2*pi
                a_v = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*j*2*pi
                a = kron_add(a_h,a_v).reshape(N*N)
                A[:,i*N_2+j] = torch.exp(1j*a)
        for i in range(M*M):
            A_t[i*N*N:(i+1)*N*N,i*N_2*N_2:(i+1)*N_2*N_2] = A
        H_RU = Channel_RIS_UE(param_list,batch,alpha)

        S_digital = torch.randn(L,Nc,4,1).cuda()+1j*torch.randn(L,Nc,4,1).cuda() #L个时隙BS导频,这里假设频率选择
        S_analog = torch.rand(L,1,M_ant,4).cuda() * 2*pi
        Phase = torch.rand(L,M_ant).cuda() * 2*pi   #L个时隙RIS相位

        Y = torch.zeros(H_RU.shape[0],Nc,L).cuda() + 0j#总观测【batch,Nc,L】

        Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j

        for l in range(L):
            s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(Nc,M_ant,1)  #导频s [Nc,M_ant]    
            
            s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
            s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(Nc))).reshape(1,1,1)
        #             print(s)
            n = (torch.randn(H_RU.shape[0],Nc) + 1j*torch.randn(H_RU.shape[0],Nc)).cuda()*sqrt(sigma/2)
            
        #             print(n)
            
            Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
            a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
            Phi[:,l,:] = a[:,0,:]
        #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
            y = (a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc) + n #[batch, Nc]
            Y[:,:,l] = y

        Theta = Phi @ A_t

        H_t_hat = BAMP_GMMV(Theta,Y.reshape(batch,Nc,L,1),K)
        H_hat = A_t @ H_t_hat
        NMSE = NMSE + ((torch.mean(torch.abs((H_RU-H_hat.reshape(batch,Nc,M_ant))*(H_RU-H_hat.reshape(batch,Nc,M_ant)))))/(torch.mean(torch.abs(H_RU*H_RU))))
    NMSE = NMSE/Step
    return H_RU,H_hat
    # print(NMSE)

def CE_AMP_GMMV(param_list,batch,pho):
#Phi 为观测矩阵 Theta为感知矩阵
    fc = param_list[0]
    BW  = param_list[1]
    Nc = param_list[2]
    M  = param_list[3]
    N  = param_list[4]
    D_sub = param_list[5]
    D_ant = param_list[6]
    R = param_list[7]
    M_ant = param_list[8]
    h1 = param_list[9]
    h2 = param_list[10]
    L =  param_list[11]
    sigma = param_list[12]
    B = param_list[13]
    K = param_list[14]
    K1 = K
    K2 = K
    N_2 = pho*N 

    alpha = 1e5/sqrt(Lp)
    # H_RU = H_RU.reshape(batch,Nc,M_ant,1)
    Step = 1


    NMSE = 0

    for step in range(Step):

        A = torch.zeros(N*N,N_2*N_2).cuda() + 0j
        A_t = torch.zeros(M_ant,N_2*N_2*M*M).cuda() + 0j
        for i in range(N_2):
            for j in range(N_2):
                a_h = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*i*2*pi
                a_v = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*j*2*pi
                a = kron_add(a_h,a_v).reshape(N*N)
                A[:,i*N_2+j] = torch.exp(1j*a)
        for i in range(M*M):
            A_t[i*N*N:(i+1)*N*N,i*N_2*N_2:(i+1)*N_2*N_2] = A

        H_RU = Channel_RIS_UE(param_list,batch,alpha)

        S_digital = torch.randn(L,Nc,4,1).cuda()+1j*torch.randn(L,Nc,4,1).cuda() #L个时隙BS导频,这里假设频率选择
        S_analog = torch.rand(L,1,M_ant,4).cuda() * 2*pi
        Phase = torch.rand(L,M_ant).cuda() * 2*pi   #L个时隙RIS相位

        Y = torch.zeros(H_RU.shape[0],Nc,L).cuda() + 0j#总观测【batch,Nc,L】

        Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j

        for l in range(L):
            s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(Nc,M_ant,1)  #导频s [Nc,M_ant]    
            
            s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
            s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(Nc))).reshape(1,1,1)
        #             print(s)
            n = (torch.randn(H_RU.shape[0],Nc) + 1j*torch.randn(H_RU.shape[0],Nc)).cuda()*sqrt(sigma/2)
            
        #             print(n)
            
            Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
            a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
            Phi[:,l,:] = a[:,0,:]
        #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
            y = (a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc) + n #[batch, Nc]
            Y[:,:,l] = y

        Theta = Phi @ A_t

        H_t_hat = AMP_GMMV(Theta,Y.reshape(batch,Nc,L,1),K)
        H_hat = A_t @ H_t_hat
        NMSE = NMSE + ((torch.mean(torch.abs((H_RU-H_hat.reshape(batch,Nc,M_ant))*(H_RU-H_hat.reshape(batch,Nc,M_ant)))))/(torch.mean(torch.abs(H_RU*H_RU))))
    NMSE = NMSE/Step
    return H_RU,H_hat
    # print(NMSE)

In [12]:
def test_channelAcquisition(param_list,batch,model_name_AE):
    
    fc = param_list[0]
#     B  = param_list1[1]
#     Nc = param_list1[2]
    M  = param_list[3]
    N  = param_list[4]
    D_sub = param_list[5]
    D_ant = param_list[6]
    R = param_list[7]
    M_ant = param_list[8]
    h1 = param_list[9]
    h2 = param_list[10]
    L =  param_list[11]
    sigma = param_list[12]
    B = param_list[13]
    Lp = param_list[15]

    net = csiNet(param_list).cuda()
    net = torch.load(model_name_AE)
        
    
    Ay = (M-1)*D_sub + (N-1)*(D_ant);#阵列长度
    Az = Ay;

    r_BS  = (torch.tensor([0,0,h1]) - torch.tensor([0,Ay/2,Az/2])).cuda()
    r_RIS = (torch.tensor([R,0,h1]) - torch.tensor([0,Ay/2,Az/2])).cuda()

    
    ##随机生成信道
    # H_BR = Channel_BS_RIS(param_list) * 1e6
    
    
    
    num_train = 0
    train_nmse = 0
    best_nmse = 1
    start = datetime.datetime.now()
    numStep = 1
    warmup_steps = 4000
    d_model = 256

    with torch.no_grad():
        net.eval()
        alpha = 1e5/sqrt(Lp)

        H_RU_SOMP = (torch.zeros(batch,Nc,M_ant) + 0j).cuda()
        H_hat_SOMP = (torch.zeros(batch,Nc,M_ant) + 0j).cuda()
        H_RU_BSOMP = (torch.zeros(batch,Nc,M_ant) + 0j).cuda()
        H_hat_BSOMP = (torch.zeros(batch,Nc,M_ant) + 0j).cuda()
        H_RU_AMP = (torch.zeros(batch,Nc,M_ant) + 0j).cuda()
        H_hat_AMP = (torch.zeros(batch,Nc,M_ant) + 0j).cuda()

        H_RU_LAMP = (torch.zeros(batch,Nc,M_ant) + 0j).cuda()
        H_hat_LAMP = (torch.zeros(batch,Nc,M_ant) + 0j).cuda()



        for ii in range(batch):

            H_RU_SOMP11,H_hat_SOMP11 = CE_SOMP_GMMV(param_list,1,4)
            H_RU_BSOMP11,H_hat_BSOMP11 = CE_BSOMP_GMMV(param_list,1,4)
            H_RU_AMP11,H_hat_AMP11 = CE_AMP_GMMV(param_list,1,2)
            

            H_RU_LAMP11,H_hat_LAMP11 = Test_LAMP_GMMV(param_list, T, 1, 1, L,sigma)

            H_hat_SOMP11 = H_hat_SOMP11.reshape(1,Nc,M_ant)
            H_hat_BSOMP11 = H_hat_BSOMP11.reshape(1,Nc,M_ant)
            H_hat_AMP11 = H_hat_AMP11.reshape(1,Nc,M_ant)

            if NMSE(H_hat_SOMP11,H_RU_SOMP11)>1:
                H_hat_SOMP11 = H_hat_SOMP11-H_hat_SOMP11
            if NMSE(H_hat_BSOMP11,H_RU_BSOMP11)>1:
                H_hat_BSOMP11 = H_hat_BSOMP11-H_hat_BSOMP11
            if NMSE(H_hat_AMP11,H_RU_AMP11)>1:
                H_hat_AMP11 = H_hat_AMP11-H_hat_AMP11
            if NMSE(H_hat_LAMP11,H_RU_LAMP11)>1:
                H_hat_LAMP11 = H_hat_LAMP11-H_hat_LAMP11

            

            H_RU_SOMP[ii,:,:] = H_RU_SOMP11[0,:,:]
            H_hat_SOMP[ii,:,:] = H_hat_SOMP11[0,:,:]
            H_RU_BSOMP[ii,:,:] = H_RU_BSOMP11[0,:,:]
            H_hat_BSOMP[ii,:,:] = H_hat_BSOMP11[0,:,:]
            H_RU_AMP[ii,:,:] = H_RU_AMP11[0,:,:]
            H_hat_AMP[ii,:,:] = H_hat_AMP11[0,:,:]
            H_RU_LAMP[ii,:,:] = H_RU_LAMP11[0,:,:]
            H_hat_LAMP[ii,:,:] = H_hat_LAMP11[0,:,:]




        H_RU_SOMP0 = net(param_list, H_BR, H_RU_SOMP)
        H_hat_SOMP0 = net(param_list, H_BR, H_hat_SOMP)
        H_hat_BSOMP0 = net(param_list, H_BR, H_hat_BSOMP)
        H_hat_AMP0 = net(param_list, H_BR, H_hat_AMP)
        H_hat_LAMP0 = net(param_list, H_BR, H_hat_LAMP)


        
        NMSE1 = NMSE(H_RU_SOMP0,H_RU_SOMP)
        NMSE2 = NMSE(H_hat_SOMP0,H_RU_SOMP)
        NMSE3 = NMSE(H_hat_BSOMP0,H_RU_BSOMP)
        NMSE4 = NMSE(H_hat_AMP0,H_RU_AMP)

        NMSE5 = NMSE(H_hat_SOMP,H_RU_SOMP)
        NMSE6 = NMSE(H_hat_BSOMP,H_RU_BSOMP)
        NMSE7 = NMSE(H_hat_AMP,H_RU_AMP)

        NMSE8 = NMSE(H_hat_LAMP,H_RU_LAMP)
        NMSE9 = NMSE(H_hat_LAMP0,H_RU_LAMP)
        print('完美信道估计+csiNet反馈 NMSE %.5f' % NMSE1.cpu()) 
        print('SOMP道估计+完美反馈 NMSE %.5f' % NMSE5.cpu())
        print('BSOMP道估计+完美反馈 NMSE %.5f' % NMSE6.cpu())
        print('AMP道估计+完美反馈 NMSE %.5f' % NMSE7.cpu())
        print('GMMV-LAMP道估计+完美反馈 NMSE %.5f' % NMSE8.cpu())

        print('SOMP道估计+csiNet反馈 NMSE %.5f' % NMSE2.cpu())
        print('BSOMP道估计+csiNet反馈 NMSE %.5f' % NMSE3.cpu())
        print('AMP道估计+csiNet反馈 NMSE %.5f' % NMSE4.cpu())
        print('GMMV-LAMP道估计+csiNet反馈 NMSE %.5f' % NMSE9.cpu())
        

#### GMMV-LAMP网络定义

In [15]:
def eta(R, sigma_t2, K, G, M, theta):  #R [batch,G,K] G为原始稀疏数据长度，K为子载波数也就是MMV的数量，M为观测维度
    theta1 = torch.abs(theta[0])
    theta2 = theta[1]
    pi_t = 1 + sigma_t2/theta1
    psi_t = 0.5*K*torch.log(torch.tensor(1+theta1/sigma_t2).cuda()) + theta2
    
#     aa = (psi_t.reshape(R.shape[0],1,1)-torch.sum(torch.abs(R*R),[2]).reshape(R.shape[0],R.shape[1],1)/((2*sigma_t2*pi_t).reshape(R.shape[0],1,1)))
#     frac = 1/torch.sigmoid(-aa)*
    
    aa = (psi_t.reshape(R.shape[0],1,1)-torch.sum(torch.abs(R*R),[2]).reshape(R.shape[0],R.shape[1],1)/((2*sigma_t2*pi_t).reshape(R.shape[0],1,1)))
    frac_div = torch.sigmoid(-aa)/pi_t.reshape(R.shape[0],1,1)


    H_res = R*frac_div
    b = torch.sum(frac_div,[1,2])/M
    return H_res,b
class LAMP_Net(nn.Module): #单层
    def __init__(self, Theta , Nc, N, L): #H [batch,G,K]为角度域信道, Theta [Nc,L,N]为感知矩阵 N为原始稀疏数据长度，T为迭代次数，Nc为子载波数也就是MMV的数量，L为观测维度
        super(LAMP_Net,self).__init__()
        self.B = torch.nn.Parameter(Hermitian(Theta))    #[Nc,N,L]
        self.theta = torch.nn.Parameter(torch.tensor([1.0,2.0]))
        
        
    def forward(self, Y, Nc, N, L, Theta): #输入Y[batch,Nc,L] 要改成[batch,L,Nc]
        
        # N = ((torch.randn(H.shape[0],M,K).cuda() + 1j*torch.randn(H.shape[0],M,K).cuda())*(sqrt(sigma2/2))).cuda()
        # Y = torch.matmul(self.A,H) + N

        Y = Y.permute(0,2,1).reshape(Y.shape[0],L,Nc)

        H_res = torch.zeros(Y.shape[0],N,Nc).cuda()
        V = Y #[batch,L,Nc]
        
        B = self.B
        theta = self.theta

        BV = B @ V.permute(0,2,1).reshape(Y.shape[0],Nc,L,1) #[batch,Nc,N,1]
        BV = BV.permute(0,2,1,3).reshape(Y.shape[0],N,Nc) #[batch,N,Nc]

        R = H_res + BV
        sigma_t2 = torch.sum(torch.abs(V*V),[1,2])/(L*Nc)
        H_res,b = eta(R, sigma_t2, Nc, N, L, theta)

        HH = Theta @ H_res.permute(0,2,1).reshape(Y.shape[0],Nc,N,1)   #[batch,Nc,L,1]
        HH = HH.permute(0,2,1,3).reshape(Y.shape[0],L,Nc) #[batch,L,Nc]

        V = Y-HH+b.reshape(-1,1,1)*V
        
        return H_res,V

class LAMP_MuNet(nn.Module): #多层
    def __init__(self, Theta , Nc, N, L, net): #H [batch,G,K]为角度域信道, Theta [Nc,L,N]为感知矩阵 N为原始稀疏数据长度，T为迭代次数，Nc为子载波数也就是MMV的数量，L为观测维度
        super(LAMP_MuNet,self).__init__()
        self.net = net
        for p in self.net.parameters():
            (p.requires_grad) = False
        
        self.B = torch.nn.Parameter(Hermitian(Theta))    #[Nc,N,L]
        self.theta = torch.nn.Parameter(torch.tensor([1.0,2.0]))
        
        
    def forward(self, Y, Nc, N, L, Theta): #输入Y[batch,Nc,L] 要改成[batch,L,Nc]

        H_res,V = self.net(Y, Nc, N, L, Theta) 

        Y = Y.permute(0,2,1).reshape(Y.shape[0],L,Nc)

        B = self.B
        theta = self.theta

        BV = B @ V.permute(0,2,1).reshape(Y.shape[0],Nc,L,1) #[batch,Nc,N,1]
        BV = BV.permute(0,2,1,3).reshape(Y.shape[0],N,Nc) #[batch,N,Nc]

        R = H_res + BV
        sigma_t2 = torch.sum(torch.abs(V*V),[1,2])/(L*Nc)
        H_res,b = eta(R, sigma_t2, Nc, N, L, theta)

        HH = Theta @ H_res.permute(0,2,1).reshape(Y.shape[0],Nc,N,1)   #[batch,Nc,L,1]
        HH = HH.permute(0,2,1,3).reshape(Y.shape[0],L,Nc) #[batch,L,Nc]

        V = Y-HH+b.reshape(-1,1,1)*V
        
        return H_res,V

In [16]:
def NMSE(H_RU_hat,H_RU):    
    NMSE = torch.sum(torch.abs((H_RU_hat-H_RU))**2)/torch.sum(torch.abs(H_RU)**2)
    return NMSE  
def CE_LAMP_GMMV(param_list, L,sigma): #K1用户数 K2稀疏度
#Phi 为观测矩阵 Theta为感知矩阵

    factor = 100000*L/32
    B = 4096
    # param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]

    alpha = 1e5
    # H_RU = H_RU.reshape(batch,Nc,M_ant,1)
    Step = 3000

    batch = 64

    S_digital = torch.randn(L,Nc,4,1).cuda()+1j*torch.randn(L,Nc,4,1).cuda() #L个时隙BS导频,这里假设频率选择
    S_analog = torch.rand(L,1,M_ant,4).cuda() * 2*pi
    Phase = torch.rand(L,M_ant).cuda() * 2*pi   #L个时隙RIS相位
    Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j
    for l in range(L):
        s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(Nc,M_ant,1)  #导频s [Nc,M_ant]    
        s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
        s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(Nc))).reshape(1,1,1)
    #             print(s)
        
    #             print(n)
        
        Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
        a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
        Phi[:,l,:] = a[:,0,:]
    #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
    Theta = Phi @ A_t
    Theta = Theta/sqrt(factor)

    for T in range(5):
        if T==0:
            net = LAMP_Net(Theta, Nc, M_ant*pho*pho, L).cuda()
        else:  
            net_1 =  torch.load('./models/GMMV_LAMP'+str(T)+'_'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')
            net = LAMP_MuNet(Theta, Nc, M_ant*pho*pho, L, net_1).cuda()
            S_digital = torch.load('./models/S_digital'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')
            S_analog  = torch.load('./models/S_analog'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')
            Phase = torch.load('./models/Phase'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')
            # net = LAMP_MuNet(Theta , Nc, M_ant*pho*pho, L, net).cuda()
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),lr=0.001)

        num_train = 0
        train_nmse = 0
        best_nmse = 2
        start = datetime.datetime.now()
        

        for step in range(Step):
            net.train()
            H_RU = Channel_RIS_UE(param_list,batch,alpha)


            Y = torch.zeros(H_RU.shape[0],Nc,L).cuda() + 0j#总观测【batch,Nc,L】

            Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j

            for l in range(L):
                s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(Nc,M_ant,1)  #导频s [Nc,M_ant]    
                
                s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
                s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(Nc))).reshape(1,1,1)
            #             print(s)
                n = (torch.randn(H_RU.shape[0],Nc) + 1j*torch.randn(H_RU.shape[0],Nc)).cuda()*sqrt(sigma/2)
                
            #             print(n)
                
                Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
                a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
                Phi[:,l,:] = a[:,0,:]
            #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
                y = (a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc) + n #[batch, Nc]
                Y[:,:,l] = y

            Theta = Phi @ A_t
            Theta = Theta/sqrt(factor)
            Y = Y/sqrt(factor)

            H_t_hat,V = net(Y, Nc, M_ant*pho*pho, L, Theta) #[batch,过采样,Nc]
            H_t_hat = H_t_hat.permute(0,2,1).reshape(Y.shape[0],Nc,M_ant*pho*pho,1)
            H_hat = A_t @ H_t_hat
            H_hat = H_hat.reshape(-1,Nc,M_ant)
            loss = NMSE(H_hat,H_RU)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            num_train = num_train + 1
            train_nmse = train_nmse + loss

            if (step+1)%100==0:
                net.eval() 
                with torch.no_grad():
                    H_RU = Channel_RIS_UE(param_list,batch,alpha)


                    Y = torch.zeros(H_RU.shape[0],Nc,L).cuda() + 0j#总观测【batch,Nc,L】

                    Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j

                    for l in range(L):
                        s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(Nc,M_ant,1)  #导频s [Nc,M_ant]    
                        
                        s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
                        s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(Nc))).reshape(1,1,1)
                    #             print(s)
                        n = (torch.randn(H_RU.shape[0],Nc) + 1j*torch.randn(H_RU.shape[0],Nc)).cuda()*sqrt(sigma/2)
                        
                    #             print(n)
                        
                        Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
                        a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
                        Phi[:,l,:] = a[:,0,:]
                    #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
                        y = (a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc) + n #[batch, Nc]
                        Y[:,:,l] = y

                    Theta = Phi @ A_t
                    Theta = Theta/sqrt(factor)
                    Y = Y/sqrt(factor)

                    H_t_hat,V = net(Y, Nc, M_ant*pho*pho, L, Theta) #[batch,过采样,Nc]
                    H_t_hat = H_t_hat.permute(0,2,1).reshape(Y.shape[0],Nc,M_ant*pho*pho,1)
                    H_hat = A_t @ H_t_hat
                    H_hat = H_hat.reshape(-1,Nc,M_ant)
                    loss = NMSE(H_hat, H_RU)

                    train_nmse = train_nmse/num_train
                    time0 =  datetime.datetime.now()-start
                    print('step:',step,'time',time0,'train NMSE %.5f' % train_nmse.cpu(),'test NMSE %.5f' % loss.cpu()) 

                    if loss < best_nmse:
                        best_nmse = loss
                        torch.save(net, './models/GMMV_LAMP'+str(T+1)+'_'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')
                        torch.save(S_digital, './models/S_digital'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')
                        torch.save(S_analog, './models/S_analog'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')
                        torch.save(Phase, './models/Phase'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')
                        print('Model saved!')
                    num_train = 0
                    train_nmse = 0
                    start = datetime.datetime.now()

In [17]:
def Test_LAMP_GMMV(param_list, T, batch, Step, L,sigma): #K1用户数 K2稀疏度
#Phi 为观测矩阵 Theta为感知矩阵

    factor = 100000*L/32
    B = 4096
    # param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]

    alpha = 1e5
    # H_RU = H_RU.reshape(batch,Nc,M_ant,1)

    batch = 64

    S_digital = torch.load('./models/S_digital'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')
    S_analog  = torch.load('./models/S_analog'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')
    Phase = torch.load('./models/Phase'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')
    Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j
    for l in range(L):
        s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(Nc,M_ant,1)  #导频s [Nc,M_ant]    
        s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
        s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(Nc))).reshape(1,1,1)
    #             print(s)
        
    #             print(n)
        
        Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
        a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
        Phi[:,l,:] = a[:,0,:]
    #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
    Theta = Phi @ A_t
    Theta = Theta/sqrt(factor)

    for TT in range(T):
        if TT==0:
            net = LAMP_Net(Theta, Nc, M_ant*pho*pho, L).cuda()
        else:  
            net = LAMP_MuNet(Theta, Nc, M_ant*pho*pho, L, net).cuda()
    net =  torch.load('./models/GMMV_LAMP'+str(T)+'_'+str(L)+'pilots_'+str(B)+'bits_'+str(Lp)+'paths_'+str(floor(log10(sigma)))+'sigma'+'.pth')

            # net = LAMP_MuNet(Theta , Nc, M_ant*pho*pho, L, net).cuda()

    

    for step in range(Step):

        net.eval() 
        with torch.no_grad():
            H_RU = Channel_RIS_UE(param_list,batch,alpha)


            Y = torch.zeros(H_RU.shape[0],Nc,L).cuda() + 0j#总观测【batch,Nc,L】

            Phi = torch.zeros(Nc,L,M_ant).cuda() + 0j

            for l in range(L):
                s = (torch.exp(1j*S_analog[l,:,:,:]) @ S_digital[l,:,:,:] ).reshape(Nc,M_ant,1)  #导频s [Nc,M_ant]    
                
                s_sigma = torch.sqrt(torch.sum(torch.abs(s*s)))
                s = s/s_sigma.reshape(1,1,1)*torch.min(s_sigma,torch.tensor(sqrt(Nc))).reshape(1,1,1)
            #             print(s)
                n = (torch.randn(H_RU.shape[0],Nc) + 1j*torch.randn(H_RU.shape[0],Nc)).cuda()*sqrt(sigma/2)
                
            #             print(n)
                
                Theta = torch.diag(torch.exp(1j*Phase[l,:])) #RIS反射矩阵【M_ant,M_ant】 
                a =  (Theta @ H_BR @ s).permute(0,2,1) #a[Nc,1,M_ant] = Theta * G * s_l
                Phi[:,l,:] = a[:,0,:]
            #             print((a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc))
                y = (a @ H_RU.reshape(-1,Nc,M_ant,1)).reshape(-1,Nc) + n #[batch, Nc]
                Y[:,:,l] = y

            Theta = Phi @ A_t
            Theta = Theta/sqrt(factor)
            Y = Y/sqrt(factor)

            H_t_hat,V = net(Y, Nc, M_ant*pho*pho, L, Theta) #[batch,过采样,Nc]
            H_t_hat = H_t_hat.permute(0,2,1).reshape(Y.shape[0],Nc,M_ant*pho*pho,1)
            H_hat = A_t @ H_t_hat
            H_hat = H_hat.reshape(-1,Nc,M_ant)

            if step == 0:
                H_RU_set = H_RU
                H_hat_set = H_hat
            else:
                H_RU_set = torch.cat([H_RU_set,H_RU],0)
                H_hat_set = torch.cat([H_hat_set,H_hat],0)
    return H_RU_set,H_hat_set




In [18]:
def test_CF_MDDL(param_list,batch,model_name_AE):
    
    fc = param_list[0]
#     B  = param_list1[1]
#     Nc = param_list1[2]
    M  = param_list[3]
    N  = param_list[4]
    D_sub = param_list[5]
    D_ant = param_list[6]
    R = param_list[7]
    M_ant = param_list[8]
    h1 = param_list[9]
    h2 = param_list[10]
    L =  param_list[11]
    sigma = param_list[12]
    B = param_list[13]
    Lp = param_list[15]

    net = csiNet(param_list).cuda()
    net = torch.load(model_name_AE)
        
    
    Ay = (M-1)*D_sub + (N-1)*(D_ant);#阵列长度
    Az = Ay;

    r_BS  = (torch.tensor([0,0,h1]) - torch.tensor([0,Ay/2,Az/2])).cuda()
    r_RIS = (torch.tensor([R,0,h1]) - torch.tensor([0,Ay/2,Az/2])).cuda()

    
    ##随机生成信道
    # H_BR = Channel_BS_RIS(param_list) * 1e6
    
    
    
    num_train = 0
    train_nmse = 0
    best_nmse = 1
    start = datetime.datetime.now()
    numStep = 1
    warmup_steps = 4000
    d_model = 256

    with torch.no_grad():
        net.eval()
        alpha = 1e5



        H_RU_LAMP = (torch.zeros(batch,Nc,M_ant) + 0j).cuda()
        H_hat_LAMP = (torch.zeros(batch,Nc,M_ant) + 0j).cuda()



        for ii in range(batch):


            
            T = 5
            H_RU_LAMP11,H_hat_LAMP11 = Test_LAMP_GMMV(param_list, T, 1, 1, L,sigma)




            if NMSE(H_hat_LAMP11,H_RU_LAMP11)>1:
                H_hat_LAMP11 = H_hat_LAMP11-H_hat_LAMP11

            

            H_RU_LAMP[ii,:,:] = H_RU_LAMP11[0,:,:]
            H_hat_LAMP[ii,:,:] = H_hat_LAMP11[0,:,:]




        H_RU_LAMP0 = net(param_list, H_BR, H_RU_LAMP)
        H_hat_LAMP0 = net(param_list, H_BR, H_hat_LAMP)


        
        NMSE1 = NMSE(H_RU_LAMP0,H_RU_LAMP)


        NMSE8 = NMSE(H_hat_LAMP,H_RU_LAMP)
        NMSE9 = NMSE(H_hat_LAMP0,H_RU_LAMP)
        print('完美信道估计+csiNet反馈 NMSE %.5f' % NMSE1.cpu()) 
        print('GMMV-LAMP道估计+完美反馈 NMSE %.5f' % NMSE8.cpu())
        print('GMMV-LAMP道估计+csiNet反馈 NMSE %.5f' % NMSE9.cpu())
        

### 训练CSINET

In [83]:
B = 8

batch = 64;   #用户数
num_batch = 1
STEPS = 10000
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
train_CF(param_list,num_batch,batch,STEPS,model_name_CF)

step: 49 time 0:00:13.818283 train NMSE 1.05149 test NMSE 1.00230
step: 99 time 0:00:13.591619 train NMSE 1.00139 test NMSE 1.00089
step: 149 time 0:00:13.683874 train NMSE 1.00060 test NMSE 1.00020
step: 199 time 0:00:13.792758 train NMSE 1.00003 test NMSE 0.99912
Model saved!
step: 249 time 0:00:13.936893 train NMSE 0.99300 test NMSE 0.98218
Model saved!
step: 299 time 0:00:14.084904 train NMSE 0.96622 test NMSE 0.97257
Model saved!
step: 349 time 0:00:14.185765 train NMSE 0.93593 test NMSE 0.93049
Model saved!
step: 399 time 0:00:14.242995 train NMSE 0.90795 test NMSE 0.88426
Model saved!
step: 449 time 0:00:14.221688 train NMSE 0.88884 test NMSE 0.88310
Model saved!
step: 499 time 0:00:14.219037 train NMSE 0.88253 test NMSE 0.86145
Model saved!
step: 549 time 0:00:14.294446 train NMSE 0.86688 test NMSE 0.84659
Model saved!
step: 599 time 0:00:14.317091 train NMSE 0.85586 test NMSE 0.86890
step: 649 time 0:00:14.311648 train NMSE 0.84578 test NMSE 0.82185
Model saved!
step: 699 time

In [84]:
B = 16

batch = 64;   #用户数
num_batch = 1
STEPS = 10000
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
train_CF(param_list,num_batch,batch,STEPS,model_name_CF)

step: 49 time 0:00:32.817091 train NMSE 1.03937 test NMSE 1.00066
step: 99 time 0:00:32.864644 train NMSE 0.99924 test NMSE 0.99973
Model saved!
step: 149 time 0:00:32.863741 train NMSE 0.99372 test NMSE 0.99121
Model saved!
step: 199 time 0:00:32.790917 train NMSE 0.96281 test NMSE 0.93677
Model saved!
step: 249 time 0:00:32.835745 train NMSE 0.89593 test NMSE 0.86316
Model saved!
step: 299 time 0:00:32.828158 train NMSE 0.83325 test NMSE 0.81329
Model saved!
step: 349 time 0:00:34.166342 train NMSE 0.80315 test NMSE 0.79293
Model saved!
step: 399 time 0:00:32.831539 train NMSE 0.78369 test NMSE 0.76983
Model saved!
step: 449 time 0:00:32.828366 train NMSE 0.77479 test NMSE 0.74959
Model saved!
step: 499 time 0:00:32.842850 train NMSE 0.75628 test NMSE 0.74660
Model saved!
step: 549 time 0:00:32.799507 train NMSE 0.74996 test NMSE 0.73093
Model saved!
step: 599 time 0:00:32.822356 train NMSE 0.73641 test NMSE 0.72133
Model saved!
step: 649 time 0:00:32.832841 train NMSE 0.72793 test N

In [17]:
B = 32

batch = 64;   #用户数
num_batch = 1
STEPS = 10000
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
train_CF(param_list,num_batch,batch,STEPS,model_name_CF)

step: 49 time 0:00:57.854232 train NMSE 1.02997 test NMSE 1.00084
step: 99 time 0:00:55.953507 train NMSE 0.99885 test NMSE 0.99408
Model saved!
step: 149 time 0:00:56.005501 train NMSE 0.96687 test NMSE 0.93665
Model saved!
step: 199 time 0:00:55.928976 train NMSE 0.86472 test NMSE 0.82075
Model saved!
step: 249 time 0:00:52.908381 train NMSE 0.79274 test NMSE 0.78855
Model saved!
step: 299 time 0:00:53.001062 train NMSE 0.76019 test NMSE 0.74179
Model saved!
step: 349 time 0:00:56.020829 train NMSE 0.73327 test NMSE 0.72676
Model saved!
step: 399 time 0:00:56.254396 train NMSE 0.70989 test NMSE 0.67509
Model saved!
step: 449 time 0:00:53.310002 train NMSE 0.67045 test NMSE 0.64165
Model saved!
step: 499 time 0:00:56.349970 train NMSE 0.62024 test NMSE 0.60026
Model saved!
step: 549 time 0:00:56.086186 train NMSE 0.58869 test NMSE 0.56823
Model saved!
step: 599 time 0:00:52.949418 train NMSE 0.55407 test NMSE 0.54256
Model saved!
step: 649 time 0:00:55.912656 train NMSE 0.53958 test N

In [18]:
B = 64

batch = 64;   #用户数
num_batch = 1
STEPS = 10000
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
train_CF(param_list,num_batch,batch,STEPS,model_name_CF)

step: 49 time 0:00:53.428184 train NMSE 1.02426 test NMSE 0.99577
Model saved!
step: 99 time 0:00:56.512791 train NMSE 0.89524 test NMSE 0.81480
Model saved!
step: 149 time 0:00:53.444332 train NMSE 0.77403 test NMSE 0.72931
Model saved!
step: 199 time 0:00:56.614122 train NMSE 0.73024 test NMSE 0.69389
Model saved!
step: 249 time 0:00:53.451454 train NMSE 0.68771 test NMSE 0.66576
Model saved!
step: 299 time 0:00:56.522473 train NMSE 0.64074 test NMSE 0.60118
Model saved!
step: 349 time 0:00:53.452244 train NMSE 0.57866 test NMSE 0.55672
Model saved!
step: 399 time 0:00:56.537518 train NMSE 0.53622 test NMSE 0.49849
Model saved!
step: 449 time 0:00:56.736795 train NMSE 0.48895 test NMSE 0.47014
Model saved!
step: 499 time 0:00:53.341505 train NMSE 0.43360 test NMSE 0.39906
Model saved!
step: 549 time 0:00:56.501018 train NMSE 0.38708 test NMSE 0.35589
Model saved!
step: 599 time 0:00:56.586394 train NMSE 0.36202 test NMSE 0.34834
Model saved!
step: 649 time 0:00:53.462579 train NMSE 0

In [19]:
B = 128

batch = 64;   #用户数
num_batch = 1
STEPS = 10000
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
train_CF(param_list,num_batch,batch,STEPS,model_name_CF)

step: 49 time 0:00:53.468635 train NMSE 1.02997 test NMSE 0.99813
Model saved!
step: 99 time 0:00:56.764568 train NMSE 0.88763 test NMSE 0.79268
Model saved!
step: 149 time 0:00:53.496301 train NMSE 0.69998 test NMSE 0.62065
Model saved!
step: 199 time 0:00:56.562249 train NMSE 0.56467 test NMSE 0.53541
Model saved!
step: 249 time 0:00:53.492162 train NMSE 0.47309 test NMSE 0.44884
Model saved!
step: 299 time 0:00:56.609471 train NMSE 0.39226 test NMSE 0.37055
Model saved!
step: 349 time 0:00:56.564930 train NMSE 0.31371 test NMSE 0.25140
Model saved!
step: 399 time 0:00:53.492155 train NMSE 0.24222 test NMSE 0.22009
Model saved!
step: 449 time 0:00:56.535993 train NMSE 0.19996 test NMSE 0.19062
Model saved!
step: 499 time 0:00:53.478099 train NMSE 0.17649 test NMSE 0.15450
Model saved!
step: 549 time 0:00:56.579389 train NMSE 0.15867 test NMSE 0.12606
Model saved!
step: 599 time 0:00:53.493465 train NMSE 0.15040 test NMSE 0.12812
step: 649 time 0:00:56.563969 train NMSE 0.13604 test N

In [17]:
B = 256

batch = 64;   #用户数
num_batch = 1
STEPS = 10000
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
train_CF(param_list,num_batch,batch,STEPS,model_name_CF)

step: 49 time 0:00:54.384621 train NMSE 1.02029 test NMSE 0.95031
Model saved!
step: 99 time 0:00:50.025462 train NMSE 0.78403 test NMSE 0.73160
Model saved!
step: 149 time 0:00:47.488076 train NMSE 0.56877 test NMSE 0.43711
Model saved!
step: 199 time 0:00:47.428806 train NMSE 0.36677 test NMSE 0.29545
Model saved!
step: 249 time 0:00:47.436331 train NMSE 0.25146 test NMSE 0.19214
Model saved!
step: 299 time 0:00:50.056980 train NMSE 0.17974 test NMSE 0.15279
Model saved!
step: 349 time 0:00:50.077803 train NMSE 0.15339 test NMSE 0.12467
Model saved!
step: 399 time 0:00:50.080745 train NMSE 0.13883 test NMSE 0.11672
Model saved!
step: 449 time 0:00:50.077856 train NMSE 0.12314 test NMSE 0.09937
Model saved!
step: 499 time 0:00:50.129297 train NMSE 0.11426 test NMSE 0.10259
step: 549 time 0:00:50.093748 train NMSE 0.10902 test NMSE 0.09393
Model saved!
step: 599 time 0:00:50.138650 train NMSE 0.09991 test NMSE 0.08748
Model saved!
step: 649 time 0:00:47.505510 train NMSE 0.09861 test N

In [17]:
BW

10000000000.0

### Test csiNet

In [ ]:
B = 64
L = 1
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04008
SOMP道估计+完美反馈 NMSE 0.99900
BSOMP道估计+完美反馈 NMSE 0.99871
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 1.09190
BSOMP道估计+csiNet反馈 NMSE 1.09106
AMP道估计+csiNet反馈 NMSE 1.08704


In [ ]:
B = 64
L = 2
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04227
SOMP道估计+完美反馈 NMSE 0.99971
BSOMP道估计+完美反馈 NMSE 0.99041
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 1.09132
BSOMP道估计+csiNet反馈 NMSE 1.07413
AMP道估计+csiNet反馈 NMSE 1.09231


In [ ]:
B = 64
L = 4
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04107
SOMP道估计+完美反馈 NMSE 1.00000
BSOMP道估计+完美反馈 NMSE 0.98601
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 1.09377
BSOMP道估计+csiNet反馈 NMSE 1.05731
AMP道估计+csiNet反馈 NMSE 1.08874


In [ ]:
B = 64
L = 8
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04044
SOMP道估计+完美反馈 NMSE 0.95751
BSOMP道估计+完美反馈 NMSE 0.45456
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 1.05400
BSOMP道估计+csiNet反馈 NMSE 0.35872
AMP道估计+csiNet反馈 NMSE 1.08972


In [ ]:
B = 64
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04553
SOMP道估计+完美反馈 NMSE 0.31151
BSOMP道估计+完美反馈 NMSE 0.13828
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 0.32229
BSOMP道估计+csiNet反馈 NMSE 0.11716
AMP道估计+csiNet反馈 NMSE 1.08931


In [ ]:
B = 64
L = 32
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04085
SOMP道估计+完美反馈 NMSE 0.11240
BSOMP道估计+完美反馈 NMSE 0.08451
AMP道估计+完美反馈 NMSE 0.13510
SOMP道估计+csiNet反馈 NMSE 0.09713
BSOMP道估计+csiNet反馈 NMSE 0.08445
AMP道估计+csiNet反馈 NMSE 0.15815


In [ ]:
B = 64
L = 64
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04205
SOMP道估计+完美反馈 NMSE 0.07626
BSOMP道估计+完美反馈 NMSE 0.06469
AMP道估计+完美反馈 NMSE 0.01724
SOMP道估计+csiNet反馈 NMSE 0.07618
BSOMP道估计+csiNet反馈 NMSE 0.07355
AMP道估计+csiNet反馈 NMSE 0.04480


In [ ]:
B = 8
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.72169
SOMP道估计+完美反馈 NMSE 0.32255
BSOMP道估计+完美反馈 NMSE 0.12935
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 0.83182
BSOMP道估计+csiNet反馈 NMSE 0.74119
AMP道估计+csiNet反馈 NMSE 1.02660


In [ ]:
B = 16
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.36793
SOMP道估计+完美反馈 NMSE 0.33611
BSOMP道估计+完美反馈 NMSE 0.12922
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 0.65856
BSOMP道估计+csiNet反馈 NMSE 0.41504
AMP道估计+csiNet反馈 NMSE 1.19024


In [ ]:
B = 32
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09593
SOMP道估计+完美反馈 NMSE 0.34198
BSOMP道估计+完美反馈 NMSE 0.12862
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 0.47044
BSOMP道估计+csiNet反馈 NMSE 0.17412
AMP道估计+csiNet反馈 NMSE 1.24075


In [ ]:
B = 64
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.03944
SOMP道估计+完美反馈 NMSE 0.31139
BSOMP道估计+完美反馈 NMSE 0.13136
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 0.31320
BSOMP道估计+csiNet反馈 NMSE 0.11795
AMP道估计+csiNet反馈 NMSE 1.08776


In [ ]:
B = 128
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.02304
SOMP道估计+完美反馈 NMSE 0.33141
BSOMP道估计+完美反馈 NMSE 0.14613
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 0.26410
BSOMP道估计+csiNet反馈 NMSE 0.08320
AMP道估计+csiNet反馈 NMSE 1.05489


In [ ]:
B = 256
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.01356
SOMP道估计+完美反馈 NMSE 0.34043
BSOMP道估计+完美反馈 NMSE 0.12739
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 0.24218
BSOMP道估计+csiNet反馈 NMSE 0.05778
AMP道估计+csiNet反馈 NMSE 1.01043


In [ ]:
B = 32
L = 16
sigma = 1e1
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09625
SOMP道估计+完美反馈 NMSE 0.30849
BSOMP道估计+完美反馈 NMSE 0.12572
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 0.48722
BSOMP道估计+csiNet反馈 NMSE 0.17967
AMP道估计+csiNet反馈 NMSE 1.22755


In [ ]:
B = 32
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09202
SOMP道估计+完美反馈 NMSE 0.31364
BSOMP道估计+完美反馈 NMSE 0.13797
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 0.43396
BSOMP道估计+csiNet反馈 NMSE 0.18291
AMP道估计+csiNet反馈 NMSE 1.24841


In [ ]:
B = 32
L = 16
sigma = 1e3
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09132
SOMP道估计+完美反馈 NMSE 0.39242
BSOMP道估计+完美反馈 NMSE 0.17413
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 0.53521
BSOMP道估计+csiNet反馈 NMSE 0.18207
AMP道估计+csiNet反馈 NMSE 1.23704


In [ ]:
B = 32
L = 16
sigma = 1e4
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09930
SOMP道估计+完美反馈 NMSE 0.87246
BSOMP道估计+完美反馈 NMSE 0.57901
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 0.95569
BSOMP道估计+csiNet反馈 NMSE 0.24812
AMP道估计+csiNet反馈 NMSE 1.23209


In [ ]:
B = 32
L = 16
sigma = 1e5
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09002
SOMP道估计+完美反馈 NMSE 1.00000
BSOMP道估计+完美反馈 NMSE 1.00000
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 1.22897
BSOMP道估计+csiNet反馈 NMSE 1.22611
AMP道估计+csiNet反馈 NMSE 1.24133


In [ ]:
B = 32
L = 16
sigma = 1e6
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09521
SOMP道估计+完美反馈 NMSE 1.00000
BSOMP道估计+完美反馈 NMSE 1.00000
AMP道估计+完美反馈 NMSE 1.00000
SOMP道估计+csiNet反馈 NMSE 1.24329
BSOMP道估计+csiNet反馈 NMSE 1.23227
AMP道估计+csiNet反馈 NMSE 1.23144


### GMMV-LAMP + csiNet

In [33]:
B = 64
L = 1
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)


完美信道估计+csiNet反馈 NMSE 0.04097
GMMV-LAMP道估计+完美反馈 NMSE 0.97291
GMMV-LAMP道估计+csiNet反馈 NMSE 1.09622


In [34]:
B = 64
L = 2
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.03944
GMMV-LAMP道估计+完美反馈 NMSE 0.96958
GMMV-LAMP道估计+csiNet反馈 NMSE 1.09345


In [35]:
B = 64
L = 4
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04249
GMMV-LAMP道估计+完美反馈 NMSE 0.75001
GMMV-LAMP道估计+csiNet反馈 NMSE 0.91940


In [36]:
B = 64
L = 8
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.03869
GMMV-LAMP道估计+完美反馈 NMSE 0.21550
GMMV-LAMP道估计+csiNet反馈 NMSE 0.13425


In [37]:
B = 64
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04075
GMMV-LAMP道估计+完美反馈 NMSE 0.11457
GMMV-LAMP道估计+csiNet反馈 NMSE 0.07496


In [38]:
B = 64
L = 32
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04108
GMMV-LAMP道估计+完美反馈 NMSE 0.03315
GMMV-LAMP道估计+csiNet反馈 NMSE 0.04758


In [39]:
B = 64
L = 64
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04124
GMMV-LAMP道估计+完美反馈 NMSE 0.01669
GMMV-LAMP道估计+csiNet反馈 NMSE 0.04282


In [40]:
B = 8
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.71706
GMMV-LAMP道估计+完美反馈 NMSE 0.12258
GMMV-LAMP道估计+csiNet反馈 NMSE 0.73390


In [41]:
B = 16
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.37059
GMMV-LAMP道估计+完美反馈 NMSE 0.12474
GMMV-LAMP道估计+csiNet反馈 NMSE 0.41655


In [42]:
B = 32
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09177
GMMV-LAMP道估计+完美反馈 NMSE 0.12206
GMMV-LAMP道估计+csiNet反馈 NMSE 0.15328


In [43]:
B = 64
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.04209
GMMV-LAMP道估计+完美反馈 NMSE 0.12145
GMMV-LAMP道估计+csiNet反馈 NMSE 0.07362


In [44]:
B = 128
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.02335
GMMV-LAMP道估计+完美反馈 NMSE 0.12628
GMMV-LAMP道估计+csiNet反馈 NMSE 0.04772


In [45]:
B = 256
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.01413
GMMV-LAMP道估计+完美反馈 NMSE 0.12149
GMMV-LAMP道估计+csiNet反馈 NMSE 0.03579


In [46]:
B = 32
L = 16
sigma = 1e1
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09706
GMMV-LAMP道估计+完美反馈 NMSE 0.06252
GMMV-LAMP道估计+csiNet反馈 NMSE 0.12598


In [47]:
B = 32
L = 16
sigma = 1e2
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.10048
GMMV-LAMP道估计+完美反馈 NMSE 0.11971
GMMV-LAMP道估计+csiNet反馈 NMSE 0.14442


In [48]:
B = 32
L = 16
sigma = 1e3
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09730
GMMV-LAMP道估计+完美反馈 NMSE 0.18183
GMMV-LAMP道估计+csiNet反馈 NMSE 0.16330


In [49]:
B = 32
L = 16
sigma = 1e4
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09158
GMMV-LAMP道估计+完美反馈 NMSE 0.46768
GMMV-LAMP道估计+csiNet反馈 NMSE 0.41230


In [50]:
B = 32
L = 16
sigma = 1e5
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.09650
GMMV-LAMP道估计+完美反馈 NMSE 0.87797
GMMV-LAMP道估计+csiNet反馈 NMSE 1.03291


In [51]:
B = 32
L = 16
sigma = 1e6
batch = 256;   #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
model_name_CF = './models/RIS_CF_'+str(B)+'bits_'+str(Lp)+'paths'+'.pth'
test_CF_MDDL(param_list,batch,model_name_CF)

完美信道估计+csiNet反馈 NMSE 0.10033
GMMV-LAMP道估计+完美反馈 NMSE 0.99690
GMMV-LAMP道估计+csiNet反馈 NMSE 1.24200


### GMMV-LAMP 训练

In [19]:
pho = 2 #过采样倍数
N_2 = pho*N 


A = torch.zeros(N*N,N_2*N_2).cuda() + 0j
A_t = torch.zeros(M_ant,N_2*N_2*M*M).cuda() + 0j
for i in range(N_2):
    for j in range(N_2):
        a_h = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*i*2*pi
        a_v = torch.range(0,N-1).reshape(1,N).cuda()/N/pho*j*2*pi
        a = kron_add(a_h,a_v).reshape(N*N)
        A[:,i*N_2+j] = torch.exp(1j*a)
for i in range(M*M):
    A_t[i*N*N:(i+1)*N*N,i*N_2*N_2:(i+1)*N_2*N_2] = A

In [20]:
L  = 1  #观测时隙数
sigma = 1e2
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:02.037092 train NMSE 10.65716 test NMSE 8.66925
step: 199 time 0:00:01.773785 train NMSE 7.54252 test NMSE 6.44187
step: 299 time 0:00:01.773853 train NMSE 5.37646 test NMSE 4.50775
step: 399 time 0:00:01.786821 train NMSE 3.93455 test NMSE 3.61277
step: 499 time 0:00:01.788264 train NMSE 2.99628 test NMSE 2.55816
step: 599 time 0:00:01.778394 train NMSE 2.37531 test NMSE 2.16657
step: 699 time 0:00:01.802039 train NMSE 1.96395 test NMSE 1.73962
Model saved!
step: 799 time 0:00:01.763833 train NMSE 1.67828 test NMSE 1.54369
Model saved!
step: 899 time 0:00:01.773304 train NMSE 1.50649 test NMSE 1.46771
Model saved!
step: 999 time 0:00:01.797696 train NMSE 1.37474 test NMSE 1.31922
Model saved!
step: 1099 time 0:00:01.791713 train NMSE 1.28245 test NMSE 1.27431
Model saved!
step: 1199 time 0:00:01.766779 train NMSE 1.21408 test NMSE 1.21348
Model saved!
step: 1299 time 0:00:01.778253 train NMSE 1.16072 test NMSE 1.15776
Model saved!
step: 1399 time 0:00:01.772258 trai

In [21]:
L  = 2  #观测时隙数
sigma = 1e2
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:01.987217 train NMSE 20.48805 test NMSE 14.95176
step: 199 time 0:00:01.963768 train NMSE 15.27663 test NMSE 13.55973
step: 299 time 0:00:01.962750 train NMSE 11.36668 test NMSE 9.97800
step: 399 time 0:00:01.956786 train NMSE 8.74011 test NMSE 7.42560
step: 499 time 0:00:01.972304 train NMSE 6.83870 test NMSE 6.46052
step: 599 time 0:00:01.958979 train NMSE 5.35346 test NMSE 4.97472
step: 699 time 0:00:01.968665 train NMSE 4.21605 test NMSE 3.40709
step: 799 time 0:00:01.953773 train NMSE 3.39833 test NMSE 3.23069
step: 899 time 0:00:01.964744 train NMSE 2.79392 test NMSE 2.43245
step: 999 time 0:00:01.976220 train NMSE 2.28228 test NMSE 2.12983
step: 1099 time 0:00:01.952070 train NMSE 1.90396 test NMSE 1.69563
Model saved!
step: 1199 time 0:00:01.958785 train NMSE 1.61969 test NMSE 1.51776
Model saved!
step: 1299 time 0:00:01.965955 train NMSE 1.42097 test NMSE 1.39534
Model saved!
step: 1399 time 0:00:01.947789 train NMSE 1.26503 test NMSE 1.18319
Model saved!
st

In [22]:
L  = 4  #观测时隙数
sigma = 1e2
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:02.328769 train NMSE 3.04297 test NMSE 2.24585
step: 199 time 0:00:02.322796 train NMSE 2.00323 test NMSE 1.63368
Model saved!
step: 299 time 0:00:02.319487 train NMSE 1.55926 test NMSE 1.36092
Model saved!
step: 399 time 0:00:02.333771 train NMSE 1.28918 test NMSE 1.20427
Model saved!
step: 499 time 0:00:02.328346 train NMSE 1.12187 test NMSE 1.03400
Model saved!
step: 599 time 0:00:02.329811 train NMSE 1.01700 test NMSE 0.99576
Model saved!
step: 699 time 0:00:02.329788 train NMSE 0.94784 test NMSE 0.97601
Model saved!
step: 799 time 0:00:02.325322 train NMSE 0.88198 test NMSE 0.87721
Model saved!
step: 899 time 0:00:02.336735 train NMSE 0.84502 test NMSE 0.83206
Model saved!
step: 999 time 0:00:02.322811 train NMSE 0.81416 test NMSE 0.79796
Model saved!
step: 1099 time 0:00:02.326745 train NMSE 0.79024 test NMSE 0.76933
Model saved!
step: 1199 time 0:00:02.322950 train NMSE 0.76853 test NMSE 0.74773
Model saved!
step: 1299 time 0:00:02.329299 train NMSE 0.75286 te

In [23]:
L  = 8  #观测时隙数
sigma = 1e2
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:03.084787 train NMSE 0.94930 test NMSE 0.87908
Model saved!
step: 199 time 0:00:03.078808 train NMSE 0.83990 test NMSE 0.78845
Model saved!
step: 299 time 0:00:03.069813 train NMSE 0.76206 test NMSE 0.72547
Model saved!
step: 399 time 0:00:03.075320 train NMSE 0.70028 test NMSE 0.66696
Model saved!
step: 499 time 0:00:03.079411 train NMSE 0.65307 test NMSE 0.64348
Model saved!
step: 599 time 0:00:03.094481 train NMSE 0.61704 test NMSE 0.57569
Model saved!
step: 699 time 0:00:03.101301 train NMSE 0.58300 test NMSE 0.56728
Model saved!
step: 799 time 0:00:03.084297 train NMSE 0.55587 test NMSE 0.56548
Model saved!
step: 899 time 0:00:03.096241 train NMSE 0.53801 test NMSE 0.52273
Model saved!
step: 999 time 0:00:03.083763 train NMSE 0.50988 test NMSE 0.49684
Model saved!
step: 1099 time 0:00:03.090745 train NMSE 0.47908 test NMSE 0.45228
Model saved!
step: 1199 time 0:00:03.078091 train NMSE 0.45149 test NMSE 0.43615
Model saved!
step: 1299 time 0:00:03.095235 train NM

In [50]:
L  = 16  #观测时隙数
sigma = 1e2
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:04.600760 train NMSE 0.87585 test NMSE 0.85593
Model saved!
step: 199 time 0:00:04.506629 train NMSE 0.77393 test NMSE 0.72020
Model saved!
step: 299 time 0:00:04.492539 train NMSE 0.68227 test NMSE 0.65930
Model saved!
step: 399 time 0:00:04.514949 train NMSE 0.62027 test NMSE 0.59680
Model saved!
step: 499 time 0:00:04.529095 train NMSE 0.58431 test NMSE 0.56634
Model saved!
step: 599 time 0:00:04.553711 train NMSE 0.55803 test NMSE 0.57227
step: 699 time 0:00:04.544385 train NMSE 0.53645 test NMSE 0.52696
Model saved!
step: 799 time 0:00:04.557846 train NMSE 0.51707 test NMSE 0.50179
Model saved!
step: 899 time 0:00:04.528928 train NMSE 0.50771 test NMSE 0.48460
Model saved!
step: 999 time 0:00:04.642258 train NMSE 0.48943 test NMSE 0.50364
step: 1099 time 0:00:04.579769 train NMSE 0.48001 test NMSE 0.45559
Model saved!
step: 1199 time 0:00:04.653627 train NMSE 0.47004 test NMSE 0.47633
step: 1299 time 0:00:04.663640 train NMSE 0.46220 test NMSE 0.44282
Model save

In [51]:
L  = 32  #观测时隙数
sigma = 1e2
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:07.677506 train NMSE 0.59002 test NMSE 0.44429
Model saved!
step: 199 time 0:00:07.652954 train NMSE 0.38871 test NMSE 0.33578
Model saved!
step: 299 time 0:00:07.736872 train NMSE 0.30523 test NMSE 0.28776
Model saved!
step: 399 time 0:00:07.681484 train NMSE 0.26723 test NMSE 0.26172
Model saved!
step: 499 time 0:00:07.661174 train NMSE 0.24056 test NMSE 0.23530
Model saved!
step: 599 time 0:00:07.669872 train NMSE 0.21825 test NMSE 0.20245
Model saved!
step: 699 time 0:00:07.709923 train NMSE 0.20254 test NMSE 0.20055
Model saved!
step: 799 time 0:00:07.648065 train NMSE 0.19189 test NMSE 0.17669
Model saved!
step: 899 time 0:00:07.662649 train NMSE 0.18295 test NMSE 0.18618
step: 999 time 0:00:07.677504 train NMSE 0.17436 test NMSE 0.18004
step: 1099 time 0:00:07.659565 train NMSE 0.16568 test NMSE 0.15169
Model saved!
step: 1199 time 0:00:07.677528 train NMSE 0.15845 test NMSE 0.16320
step: 1299 time 0:00:07.675433 train NMSE 0.15396 test NMSE 0.14672
Model save

In [29]:
L  = 64  #观测时隙数
sigma = 1e2
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:13.988841 train NMSE 0.22225 test NMSE 0.15287
Model saved!
step: 199 time 0:00:13.876553 train NMSE 0.13638 test NMSE 0.12155
Model saved!
step: 299 time 0:00:13.917938 train NMSE 0.10989 test NMSE 0.10457
Model saved!
step: 399 time 0:00:13.974770 train NMSE 0.09774 test NMSE 0.08955
Model saved!
step: 499 time 0:00:13.937353 train NMSE 0.08947 test NMSE 0.09227
step: 599 time 0:00:13.915989 train NMSE 0.08534 test NMSE 0.07588
Model saved!
step: 699 time 0:00:13.888962 train NMSE 0.08117 test NMSE 0.08396
step: 799 time 0:00:13.926921 train NMSE 0.07895 test NMSE 0.07348
Model saved!
step: 899 time 0:00:13.863019 train NMSE 0.07649 test NMSE 0.07576
step: 999 time 0:00:13.876021 train NMSE 0.07530 test NMSE 0.07103
Model saved!
step: 1099 time 0:00:13.920522 train NMSE 0.07297 test NMSE 0.07798
step: 1199 time 0:00:13.905488 train NMSE 0.07215 test NMSE 0.07288
step: 1299 time 0:00:13.878123 train NMSE 0.07073 test NMSE 0.07176
step: 1399 time 0:00:13.891640 train

In [24]:
L  = 16  #观测时隙数
sigma = 1e1
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:04.595704 train NMSE 0.78027 test NMSE 0.68867
Model saved!
step: 199 time 0:00:04.559801 train NMSE 0.62366 test NMSE 0.58220
Model saved!
step: 299 time 0:00:04.574761 train NMSE 0.55053 test NMSE 0.52232
Model saved!
step: 399 time 0:00:04.578750 train NMSE 0.49761 test NMSE 0.47654
Model saved!
step: 499 time 0:00:04.567780 train NMSE 0.46112 test NMSE 0.42281
Model saved!
step: 599 time 0:00:04.581742 train NMSE 0.43270 test NMSE 0.42757
step: 699 time 0:00:04.566783 train NMSE 0.40850 test NMSE 0.38997
Model saved!
step: 799 time 0:00:04.582752 train NMSE 0.38754 test NMSE 0.37850
Model saved!
step: 899 time 0:00:04.565800 train NMSE 0.37074 test NMSE 0.37318
Model saved!
step: 999 time 0:00:04.575773 train NMSE 0.35428 test NMSE 0.34035
Model saved!
step: 1099 time 0:00:04.561806 train NMSE 0.33918 test NMSE 0.33332
Model saved!
step: 1199 time 0:00:04.557806 train NMSE 0.32254 test NMSE 0.30660
Model saved!
step: 1299 time 0:00:04.556346 train NMSE 0.30551 te

In [25]:
L  = 16  #观测时隙数
sigma = 1e3
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:04.576362 train NMSE 0.83401 test NMSE 0.80441
Model saved!
step: 199 time 0:00:04.570296 train NMSE 0.74328 test NMSE 0.70490
Model saved!
step: 299 time 0:00:04.565856 train NMSE 0.64980 test NMSE 0.60753
Model saved!
step: 399 time 0:00:04.569905 train NMSE 0.59878 test NMSE 0.59611
Model saved!
step: 499 time 0:00:04.553249 train NMSE 0.55729 test NMSE 0.53690
Model saved!
step: 599 time 0:00:04.555369 train NMSE 0.53029 test NMSE 0.57681
step: 699 time 0:00:04.539899 train NMSE 0.51223 test NMSE 0.47126
Model saved!
step: 799 time 0:00:04.548173 train NMSE 0.49248 test NMSE 0.50415
step: 899 time 0:00:04.545377 train NMSE 0.47818 test NMSE 0.49104
step: 999 time 0:00:04.560126 train NMSE 0.46301 test NMSE 0.44418
Model saved!
step: 1099 time 0:00:04.560339 train NMSE 0.45366 test NMSE 0.42230
Model saved!
step: 1199 time 0:00:04.561838 train NMSE 0.43613 test NMSE 0.41819
Model saved!
step: 1299 time 0:00:04.613711 train NMSE 0.41775 test NMSE 0.42166
step: 1399

In [26]:
L  = 16  #观测时隙数
sigma = 1e4
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:04.577830 train NMSE 0.98112 test NMSE 0.96433
Model saved!
step: 199 time 0:00:04.556846 train NMSE 0.94311 test NMSE 0.91255
Model saved!
step: 299 time 0:00:04.565869 train NMSE 0.86525 test NMSE 0.83669
Model saved!
step: 399 time 0:00:04.554872 train NMSE 0.81526 test NMSE 0.81008
Model saved!
step: 499 time 0:00:04.546963 train NMSE 0.78163 test NMSE 0.75527
Model saved!
step: 599 time 0:00:04.563331 train NMSE 0.75527 test NMSE 0.73259
Model saved!
step: 699 time 0:00:04.568834 train NMSE 0.72982 test NMSE 0.71661
Model saved!
step: 799 time 0:00:04.550359 train NMSE 0.71234 test NMSE 0.69036
Model saved!
step: 899 time 0:00:04.563563 train NMSE 0.70155 test NMSE 0.72540
step: 999 time 0:00:04.550943 train NMSE 0.69179 test NMSE 0.70176
step: 1099 time 0:00:04.548527 train NMSE 0.68119 test NMSE 0.67474
Model saved!
step: 1199 time 0:00:04.564854 train NMSE 0.67095 test NMSE 0.68982
step: 1299 time 0:00:04.565321 train NMSE 0.66181 test NMSE 0.66565
Model save

In [27]:
L  = 16  #观测时隙数
sigma = 1e5
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:04.600764 train NMSE 0.99639 test NMSE 0.99236
Model saved!
step: 199 time 0:00:04.580816 train NMSE 0.94639 test NMSE 0.91903
Model saved!
step: 299 time 0:00:04.554826 train NMSE 0.90981 test NMSE 0.91951
step: 399 time 0:00:04.551852 train NMSE 0.90571 test NMSE 0.90035
Model saved!
step: 499 time 0:00:04.560811 train NMSE 0.90058 test NMSE 0.90788
step: 599 time 0:00:04.567811 train NMSE 0.90023 test NMSE 0.88766
Model saved!
step: 699 time 0:00:04.572788 train NMSE 0.89711 test NMSE 0.88844
step: 799 time 0:00:04.559849 train NMSE 0.89422 test NMSE 0.89237
step: 899 time 0:00:04.543393 train NMSE 0.89349 test NMSE 0.89104
step: 999 time 0:00:04.547884 train NMSE 0.89168 test NMSE 0.89503
step: 1099 time 0:00:04.567818 train NMSE 0.88880 test NMSE 0.90385
step: 1199 time 0:00:04.553846 train NMSE 0.88786 test NMSE 0.90355
step: 1299 time 0:00:04.574312 train NMSE 0.88872 test NMSE 0.88799
step: 1399 time 0:00:04.606758 train NMSE 0.88678 test NMSE 0.88596
Model s

In [28]:
L  = 16  #观测时隙数
sigma = 1e6
K = 4 #用户数
param_list = [fc,BW,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,sigma,B,K,Lp]
CE_LAMP_GMMV(param_list,L,sigma)

step: 99 time 0:00:04.591717 train NMSE 1.00459 test NMSE 1.00118
Model saved!
step: 199 time 0:00:04.584746 train NMSE 1.00024 test NMSE 0.99951
Model saved!
step: 299 time 0:00:04.564863 train NMSE 0.99878 test NMSE 0.99866
Model saved!
step: 399 time 0:00:04.559880 train NMSE 0.99806 test NMSE 0.99760
Model saved!
step: 499 time 0:00:04.567338 train NMSE 0.99758 test NMSE 0.99818
step: 599 time 0:00:04.568813 train NMSE 0.99728 test NMSE 0.99694
Model saved!
step: 699 time 0:00:04.562345 train NMSE 0.99709 test NMSE 0.99732
step: 799 time 0:00:04.574841 train NMSE 0.99705 test NMSE 0.99699
step: 899 time 0:00:04.559384 train NMSE 0.99701 test NMSE 0.99680
Model saved!
step: 999 time 0:00:04.555860 train NMSE 0.99695 test NMSE 0.99697
step: 1099 time 0:00:04.561350 train NMSE 0.99689 test NMSE 0.99706
step: 1199 time 0:00:04.557892 train NMSE 0.99681 test NMSE 0.99691
step: 1299 time 0:00:04.548922 train NMSE 0.99685 test NMSE 0.99659
Model saved!
step: 1399 time 0:00:04.559834 train